In [155]:
import pandas as pd
import duckdb
from itertools import combinations
import numpy as np

# Tables

In [3]:
patients = pd.read_csv(
    "/Users/elnurahadov/Desktop/Medical Tourism Dataset /patienten.csv",
    parse_dates=["Anmeldedatum"],
)

patients.head()

,Patienten_ID,Vorname,Nachname,Herkunftsland,Alter_bei_Anmeldung,Geschlecht,Anmeldedatum,Sprache,Vorherige_Behandlung_Deutschland,Versicherungstyp,Familienanamnese,Familienstand,Bildungsgrad,Berufsstatus
0,P00001,Anna,Lebedev,Russland,13,W,2015-03-13,Russisch,Ja,Privat,Autoimmunerkrankungen,Ledig,Grundschule,Angestellt
1,P00002,Denis,Popov,Moldawien,74,M,2015-11-29,Russisch,Nein,Gesetzlich,Erbliche Stoffwechselerkrankungen,Geschieden,Grundschule,Selbststaendig
2,P00003,Marina,Popov,Belarus,53,W,2015-06-22,Russisch,Ja,Privat,Autoimmunerkrankungen,Ledig,Abitur,Rentner
3,P00004,Vera,Kozlov,Ukraine,15,W,2015-07-13,Russisch,Ja,Ausland,Neurologische Erkrankungen,Verheiratet,Grundschule,Angestellt
4,P00005,Igor,Kozlov,Russland,47,M,2015-07-14,Russisch,Nein,Selbstzahler,Neurologische Erkrankungen,Verheiratet,Abitur,Rentner


In [4]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12200 entries, 0 to 12199
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Patienten_ID                      12200 non-null  object        
 1   Vorname                           12200 non-null  object        
 2   Nachname                          12200 non-null  object        
 3   Herkunftsland                     12200 non-null  object        
 4   Alter_bei_Anmeldung               12200 non-null  int64         
 5   Geschlecht                        12200 non-null  object        
 6   Anmeldedatum                      12200 non-null  datetime64[ns]
 7   Sprache                           12200 non-null  object        
 8   Vorherige_Behandlung_Deutschland  12200 non-null  object        
 9   Versicherungstyp                  12200 non-null  object        
 10  Familienanamnese                  12200 non-nu

In [5]:
services = pd.read_csv(
    "/Users/elnurahadov/Desktop/Medical Tourism Dataset /medizinische_leistungen.csv"
)
services.head()

,Leistungs_ID,Leistungsname,Kategorie,Kosten_EUR,Dauer_Stunden,Behandlungsart,Nachsorge_erforderlich,Qualitaetsstufe,Komplexitaet,Narkose_erforderlich,Nachsorge_Tage
0,L00001,Echokardiographie,Kardiologie,27089,4,Tagesklinik,Nein,Premium,Hoch,Ja,15
1,L00002,Herzkatheteruntersuchung,Kardiologie,20740,6,Ambulant,Nein,Premium,Niedrig,Ja,4
2,L00003,Bypass-Operation,Kardiologie,24148,7,Ambulant,Ja,Standard,Mittel,Ja,14
3,L00004,Stent-Implantation,Kardiologie,21151,12,Ambulant,Nein,Premium,Niedrig,Ja,3
4,L00005,Herzschrittmacher-Implantation,Kardiologie,29066,9,Stationaer,Ja,Premium,Hoch,Nein,29


In [6]:
services.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Leistungs_ID            80 non-null     object
 1   Leistungsname           80 non-null     object
 2   Kategorie               80 non-null     object
 3   Kosten_EUR              80 non-null     int64 
 4   Dauer_Stunden           80 non-null     int64 
 5   Behandlungsart          80 non-null     object
 6   Nachsorge_erforderlich  80 non-null     object
 7   Qualitaetsstufe         80 non-null     object
 8   Komplexitaet            80 non-null     object
 9   Narkose_erforderlich    80 non-null     object
 10  Nachsorge_Tage          80 non-null     int64 
dtypes: int64(3), object(8)
memory usage: 7.0+ KB


In [100]:
accompanying_staff = pd.read_csv(
    "/Users/elnurahadov/Desktop/Medical Tourism Dataset /begleitpersonal.csv"
)
accompanying_staff.head()

,Mitarbeiter_ID,Vorname,Nachname,Geschlecht,Sprachkombination,Verfuegbarkeit,Stundenlohn_EUR,Status,Einstellungsjahr,Berufserfahrung_Jahre,Qualifikation,Fuehrerscheinklasse,Medizinische_Grundkenntnisse,Patienten_pro_Jahr,Durchschnittsbewertung
0,M001,Anna,Mueller,W,Russisch-Deutsch,Teilzeit,53,Aktiv,2020,4,Medizinischer Uebersetzer,B,Nein,193,4.346110
1,M002,Dmitri,Schmidt,M,Russisch-Deutsch,Vollzeit,25,Aktiv,2014,10,Medizinischer Uebersetzer,C1,Ja,247,4.276325
2,M003,Elena,Weber,W,Russisch-Deutsch,Vollzeit,50,Aktiv,2009,15,Staatlich gepruefter Dolmetscher,C1,Ja,334,4.664546
3,M004,Alexander,Wagner,M,Russisch-Deutsch,Vollzeit,48,Aktiv,2017,7,Kultureller Vermittler,C2,Nein,258,4.708834
4,M005,Svetlana,Fischer,W,Russisch-Deutsch,Teilzeit,46,Aktiv,2020,4,Medizinischer Uebersetzer,C1,Ja,198,4.732080


In [101]:
accompanying_staff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Mitarbeiter_ID                10 non-null     object 
 1   Vorname                       10 non-null     object 
 2   Nachname                      10 non-null     object 
 3   Geschlecht                    10 non-null     object 
 4   Sprachkombination             10 non-null     object 
 5   Verfuegbarkeit                10 non-null     object 
 6   Stundenlohn_EUR               10 non-null     int64  
 7   Status                        10 non-null     object 
 8   Einstellungsjahr              10 non-null     int64  
 9   Berufserfahrung_Jahre         10 non-null     int64  
 10  Qualifikation                 10 non-null     object 
 11  Fuehrerscheinklasse           10 non-null     object 
 12  Medizinische_Grundkenntnisse  10 non-null     object 
 13  Patiente

In [108]:
treatment = pd.read_csv(
    "/Users/elnurahadov/Desktop/Medical Tourism Dataset /behandlungsverlaeufe.csv",
    parse_dates=[
        "Anfragedatum",
        "Beratungsdatum",
        "Behandlungsdatum",
        "Abschlussdatum",
    ],
)
treatment.head()

,Behandlungs_ID,Patienten_ID,Leistungs_ID,Mitarbeiter_ID,Anfragedatum,Beratungsdatum,Behandlungsdatum,Abschlussdatum,Status,Behandlungsort,...,Dolmetscherstunden,Leistungskosten_EUR,Dolmetscherkosten_EUR,Transportkosten_EUR,Unterkunftskosten_EUR,Zusatzkosten_EUR,Gesamtkosten_EUR,Komplikationen,Patientenzufriedenheit,Nachsorge_erforderlich
0,B00001,P00052,L00067,M009,2015-12-03,2015-12-08,2016-01-26,2016-02-18,Behandlung_abgeschlossen,Duisburg,...,19,34215,1481,513,1247,1669,39125,Nein,Neutral,Teilweise
1,B00002,P00498,L00079,M006,2015-12-23,2016-01-05,2016-02-04,2016-02-24,Abgeschlossen,Hamburg,...,3,5573,383,55,2290,1372,9673,Ja,Unzufrieden,Ja
2,B00003,P00494,L00046,M004,2015-08-19,2015-08-29,2015-10-04,2015-11-01,Anreise_geplant,Nuremberg,...,2,21937,1272,684,1988,197,26078,Nein,Zufrieden,Teilweise
3,B00004,P00298,L00053,M003,2015-05-18,2015-05-29,2015-07-13,2015-08-01,Nachsorge,Munich,...,19,44871,544,148,2326,927,48816,Nein,Zufrieden,Ja
4,B00005,P00324,L00073,M008,2015-12-05,2015-12-10,2016-01-09,2016-01-19,Termin_vereinbart,Hannover,...,15,38498,1432,253,2049,626,42858,Nein,Unzufrieden,Nein


In [102]:
treatment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8750 entries, 0 to 8749
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Behandlungs_ID          8750 non-null   object        
 1   Patienten_ID            8750 non-null   object        
 2   Leistungs_ID            8750 non-null   object        
 3   Mitarbeiter_ID          8750 non-null   object        
 4   Anfragedatum            8750 non-null   datetime64[ns]
 5   Beratungsdatum          8750 non-null   datetime64[ns]
 6   Behandlungsdatum        8750 non-null   datetime64[ns]
 7   Abschlussdatum          8750 non-null   datetime64[ns]
 8   Status                  8750 non-null   object        
 9   Behandlungsort          8750 non-null   object        
 10  Transportart            8750 non-null   object        
 11  Begleitungstage         8750 non-null   int64         
 12  Dolmetscherstunden      8750 non-null   int64   

In [103]:
accommodations = pd.read_csv(
    "/Users/elnurahadov/Desktop/Medical Tourism Dataset /unterkuenfte.csv"
)
accommodations.head()

,Unterkunft_ID,Name,Stadt,Typ,Sterne,Preis_pro_Nacht_EUR,Medizinische_Betreuung,Russischsprachiges_Personal,Transfer_Service,Kuechenzeile_verfuegbar,Anzahl_Zimmer
0,U001,Hotel Adlon Berlin,Berlin,Hotel,5,159,Ja,Ja,Nein,Ja,50
1,U002,Kempinski Munich,Munich,Hotel,5,160,Ja,Ja,Nein,Nein,26
2,U003,Steigenberger Hamburg,Hamburg,Hotel,4,333,Ja,Nein,Nein,Ja,20
3,U004,Medical Apartment Berlin,Berlin,Apartment,3,105,Ja,Ja,Nein,Nein,62
4,U005,Guesthouse Frankfurt,Frankfurt,Pension,3,182,Nein,Ja,Ja,Ja,20


In [104]:
accommodations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Unterkunft_ID                10 non-null     object
 1   Name                         10 non-null     object
 2   Stadt                        10 non-null     object
 3   Typ                          10 non-null     object
 4   Sterne                       10 non-null     int64 
 5   Preis_pro_Nacht_EUR          10 non-null     int64 
 6   Medizinische_Betreuung       10 non-null     object
 7   Russischsprachiges_Personal  10 non-null     object
 8   Transfer_Service             10 non-null     object
 9   Kuechenzeile_verfuegbar      10 non-null     object
 10  Anzahl_Zimmer                10 non-null     int64 
dtypes: int64(3), object(8)
memory usage: 1012.0+ bytes


In [13]:
reports = pd.read_csv(
    "/Users/elnurahadov/Desktop/Medical Tourism Dataset /medizinische_berichte.csv",
    parse_dates=["Erstellungsdatum"],
)
reports.head()

,Bericht_ID,Behandlungs_ID,Berichtstyp,Erstellungsdatum,Sprache,Format,An_Patient_gesendet,Befund_Status
0,R00001,B00409,Laborergebnisse,2020-03-23,Englisch,DOC,Ja,Auffaellig
1,R00002,B04297,Radiologischer_Befund,2015-12-18,Russisch,Papier,Nein,Auffaellig
2,R00003,B00426,Nachsorgeplan,2016-06-10,Deutsch,PDF,Nein,Normal
3,R00004,B08639,Radiologischer_Befund,2016-12-22,Deutsch,Papier,Nein,Normal
4,R00005,B02162,Radiologischer_Befund,2021-02-02,Deutsch,Papier,Nein,Auffaellig


In [105]:
reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Bericht_ID           5000 non-null   object        
 1   Behandlungs_ID       5000 non-null   object        
 2   Berichtstyp          5000 non-null   object        
 3   Erstellungsdatum     5000 non-null   datetime64[ns]
 4   Sprache              5000 non-null   object        
 5   Format               5000 non-null   object        
 6   An_Patient_gesendet  5000 non-null   object        
 7   Befund_Status        5000 non-null   object        
dtypes: datetime64[ns](1), object(7)
memory usage: 312.6+ KB


# Queries

## Basic Level

1. Select all patients / Alle Patienten auswählen

In [15]:
con = duckdb.connect()
con.register("Medical Tourism", patients)

result = con.execute(
    """
      SELECT *
      FROM patients    
    """
).df()

display(result)


,Patienten_ID,Vorname,Nachname,Herkunftsland,Alter_bei_Anmeldung,Geschlecht,Anmeldedatum,Sprache,Vorherige_Behandlung_Deutschland,Versicherungstyp,Familienanamnese,Familienstand,Bildungsgrad,Berufsstatus
0,P00001,Anna,Lebedev,Russland,13,W,2015-03-13,Russisch,Ja,Privat,Autoimmunerkrankungen,Ledig,Grundschule,Angestellt
1,P00002,Denis,Popov,Moldawien,74,M,2015-11-29,Russisch,Nein,Gesetzlich,Erbliche Stoffwechselerkrankungen,Geschieden,Grundschule,Selbststaendig
2,P00003,Marina,Popov,Belarus,53,W,2015-06-22,Russisch,Ja,Privat,Autoimmunerkrankungen,Ledig,Abitur,Rentner
3,P00004,Vera,Kozlov,Ukraine,15,W,2015-07-13,Russisch,Ja,Ausland,Neurologische Erkrankungen,Verheiratet,Grundschule,Angestellt
4,P00005,Igor,Kozlov,Russland,47,M,2015-07-14,Russisch,Nein,Selbstzahler,Neurologische Erkrankungen,Verheiratet,Abitur,Rentner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12195,P12196,Lyudmila,Kozlov,Russland,31,W,2024-09-25,Russisch,Nein,Gesetzlich,Herz-Kreislauf-Erkrankungen,Geschieden,Hochschulabschluss,Rentner
12196,P12197,Alla,Petrov,Kasachstan,46,W,2024-04-08,Russisch,Nein,Ausland,Herz-Kreislauf-Erkrankungen,Verwitwet,Mittlere Reife,Angestellt
12197,P12198,Galina,Lebedev,Russland,64,W,2024-09-25,Russisch,Nein,Ausland,Autoimmunerkrankungen,Verheiratet,Mittlere Reife,Angestellt
12198,P12199,Galina,Morozov,Russland,83,W,2024-01-16,Russisch,Nein,Selbstzahler,Diabetes mellitus,Verheiratet,Abitur,Selbststaendig


In [16]:
patients.head()

,Patienten_ID,Vorname,Nachname,Herkunftsland,Alter_bei_Anmeldung,Geschlecht,Anmeldedatum,Sprache,Vorherige_Behandlung_Deutschland,Versicherungstyp,Familienanamnese,Familienstand,Bildungsgrad,Berufsstatus
0,P00001,Anna,Lebedev,Russland,13,W,2015-03-13,Russisch,Ja,Privat,Autoimmunerkrankungen,Ledig,Grundschule,Angestellt
1,P00002,Denis,Popov,Moldawien,74,M,2015-11-29,Russisch,Nein,Gesetzlich,Erbliche Stoffwechselerkrankungen,Geschieden,Grundschule,Selbststaendig
2,P00003,Marina,Popov,Belarus,53,W,2015-06-22,Russisch,Ja,Privat,Autoimmunerkrankungen,Ledig,Abitur,Rentner
3,P00004,Vera,Kozlov,Ukraine,15,W,2015-07-13,Russisch,Ja,Ausland,Neurologische Erkrankungen,Verheiratet,Grundschule,Angestellt
4,P00005,Igor,Kozlov,Russland,47,M,2015-07-14,Russisch,Nein,Selbstzahler,Neurologische Erkrankungen,Verheiratet,Abitur,Rentner


2. Count total number of patients / Gesamtzahl der Patienten zählen

In [17]:
result = con.execute(
    """
      SELECT 
        COUNT(DISTINCT Patienten_ID) AS Pat_Gesamtmzahl
      FROM patients    
    """
).df()

display(result)

,Pat_Gesamtmzahl
0,12200


In [18]:
len(patients)

12200

In [19]:
patients.shape[0]

12200

3. Show all medical services / Alle medizinischen Leistungen anzeigen

In [20]:
result = con.execute(
    """
    SELECT DISTINCT Leistungsname AS Leistungsname
    FROM services
    """
).df()
display(result)

,Leistungsname
0,Herzschrittmacher-Implantation
1,Herzklappen-Operation
2,Belastungs-EKG
3,Langzeit-EKG
4,Kreuzband-Rekonstruktion
...,...
75,Netzhaut-Operation
76,Tumormarker
77,Roentgen Thorax
78,Angiographie


In [21]:
services.head()

,Leistungs_ID,Leistungsname,Kategorie,Kosten_EUR,Dauer_Stunden,Behandlungsart,Nachsorge_erforderlich,Qualitaetsstufe,Komplexitaet,Narkose_erforderlich,Nachsorge_Tage
0,L00001,Echokardiographie,Kardiologie,27089,4,Tagesklinik,Nein,Premium,Hoch,Ja,15
1,L00002,Herzkatheteruntersuchung,Kardiologie,20740,6,Ambulant,Nein,Premium,Niedrig,Ja,4
2,L00003,Bypass-Operation,Kardiologie,24148,7,Ambulant,Ja,Standard,Mittel,Ja,14
3,L00004,Stent-Implantation,Kardiologie,21151,12,Ambulant,Nein,Premium,Niedrig,Ja,3
4,L00005,Herzschrittmacher-Implantation,Kardiologie,29066,9,Stationaer,Ja,Premium,Hoch,Nein,29


4. List all staff members / Alle Mitarbeiter auflisten

In [22]:
con.execute(
    """
    SELECT
        DISTINCT Vorname, Nachname, Status 
    FROM accompanying_staff 
    """
).df()

,Vorname,Nachname,Status
0,Anna,Mueller,Aktiv
1,Irina,Schaefer,Aktiv
2,Alexander,Wagner,Aktiv
3,Dmitri,Schmidt,Aktiv
4,Sergei,Koch,Aktiv
5,Elena,Weber,Aktiv
6,Svetlana,Fischer,Aktiv
7,Marina,Schulz,Aktiv
8,Viktor,Hoffmann,Aktiv
9,Pavel,Becker,Aktiv


In [23]:
accompanying_staff[["Vorname", "Nachname", "Status"]]

,Vorname,Nachname,Status
0,Anna,Mueller,Aktiv
1,Dmitri,Schmidt,Aktiv
2,Elena,Weber,Aktiv
3,Alexander,Wagner,Aktiv
4,Svetlana,Fischer,Aktiv
5,Pavel,Becker,Aktiv
6,Marina,Schulz,Aktiv
7,Viktor,Hoffmann,Aktiv
8,Irina,Schaefer,Aktiv
9,Sergei,Koch,Aktiv


5. Show patients from Russia / Patienten aus Russland anzeigen

In [24]:
con.execute(
    """
    SELECT
        DISTINCT Vorname, Nachname
    FROM patients
    WHERE Herkunftsland = 'Russland' 
    """
).df()

,Vorname,Nachname
0,Lyudmila,Petrov
1,Igor,Stepanov
2,Tatiana,Volkov
3,Alla,Andreev
4,Sergei,Mikhailov
...,...,...
793,Irina,Yakovlev
794,Mikhail,Lebedev
795,Ekaterina,Morozov
796,Anna,Kozlov


In [25]:
patients[patients["Herkunftsland"] == "Russland"][
    ["Patienten_ID", "Vorname", "Nachname"]
].drop_duplicates()

,Patienten_ID,Vorname,Nachname
0,P00001,Anna,Lebedev
4,P00005,Igor,Kozlov
5,P00006,Kirill,Smirnov
6,P00007,Igor,Ivanov
10,P00011,Larisa,Smirnov
...,...,...,...
12191,P12192,Roman,Novikov
12194,P12195,Valentina,Makarov
12195,P12196,Lyudmila,Kozlov
12197,P12198,Galina,Lebedev


6. Find patients older than 50 / Patienten über 50 Jahre finden

In [26]:
con.execute(
    """
    SELECT
        Vorname, Nachname, Alter_bei_Anmeldung
    FROM patients
    WHERE Alter_bei_Anmeldung > 50 
    """
).df()

,Vorname,Nachname,Alter_bei_Anmeldung
0,Denis,Popov,74
1,Marina,Popov,53
2,Igor,Ivanov,51
3,Marina,Novikov,60
4,Vera,Petrov,69
...,...,...,...
5644,Natasha,Sergeev,68
5645,Artem,Sidorov,65
5646,Valentina,Makarov,76
5647,Galina,Lebedev,64


In [27]:
patients["Patienten_ID"].is_unique

True

In [28]:
patients[patients["Alter_bei_Anmeldung"] > 50][
    ["Vorname", "Nachname", "Alter_bei_Anmeldung"]
]

,Vorname,Nachname,Alter_bei_Anmeldung
1,Denis,Popov,74
2,Marina,Popov,53
6,Igor,Ivanov,51
7,Marina,Novikov,60
9,Vera,Petrov,69
...,...,...,...
12192,Natasha,Sergeev,68
12193,Artem,Sidorov,65
12194,Valentina,Makarov,76
12197,Galina,Lebedev,64


7. Show all cardiology services / Alle kardiologischen Leistungen anzeigen

In [29]:
con.execute(
    """
    SELECT
        *
    FROM services
    WHERE Kategorie = 'Kardiologie'
    """
).df()

,Leistungs_ID,Leistungsname,Kategorie,Kosten_EUR,Dauer_Stunden,Behandlungsart,Nachsorge_erforderlich,Qualitaetsstufe,Komplexitaet,Narkose_erforderlich,Nachsorge_Tage
0,L00001,Echokardiographie,Kardiologie,27089,4,Tagesklinik,Nein,Premium,Hoch,Ja,15
1,L00002,Herzkatheteruntersuchung,Kardiologie,20740,6,Ambulant,Nein,Premium,Niedrig,Ja,4
2,L00003,Bypass-Operation,Kardiologie,24148,7,Ambulant,Ja,Standard,Mittel,Ja,14
3,L00004,Stent-Implantation,Kardiologie,21151,12,Ambulant,Nein,Premium,Niedrig,Ja,3
4,L00005,Herzschrittmacher-Implantation,Kardiologie,29066,9,Stationaer,Ja,Premium,Hoch,Nein,29
5,L00006,Defibrillator-Implantation,Kardiologie,36663,12,Tagesklinik,Nein,Premium,Hoch,Nein,23
6,L00007,Herzklappen-Operation,Kardiologie,43175,10,Tagesklinik,Ja,Standard,Niedrig,Ja,25
7,L00008,Koronarangiographie,Kardiologie,34939,12,Stationaer,Nein,Luxus,Niedrig,Ja,2
8,L00009,Belastungs-EKG,Kardiologie,33435,5,Ambulant,Ja,Premium,Hoch,Ja,16
9,L00010,Langzeit-EKG,Kardiologie,8329,11,Stationaer,Nein,Luxus,Niedrig,Nein,1


In [30]:
services[services["Kategorie"] == "Kardiologie"]

,Leistungs_ID,Leistungsname,Kategorie,Kosten_EUR,Dauer_Stunden,Behandlungsart,Nachsorge_erforderlich,Qualitaetsstufe,Komplexitaet,Narkose_erforderlich,Nachsorge_Tage
0,L00001,Echokardiographie,Kardiologie,27089,4,Tagesklinik,Nein,Premium,Hoch,Ja,15
1,L00002,Herzkatheteruntersuchung,Kardiologie,20740,6,Ambulant,Nein,Premium,Niedrig,Ja,4
2,L00003,Bypass-Operation,Kardiologie,24148,7,Ambulant,Ja,Standard,Mittel,Ja,14
3,L00004,Stent-Implantation,Kardiologie,21151,12,Ambulant,Nein,Premium,Niedrig,Ja,3
4,L00005,Herzschrittmacher-Implantation,Kardiologie,29066,9,Stationaer,Ja,Premium,Hoch,Nein,29
5,L00006,Defibrillator-Implantation,Kardiologie,36663,12,Tagesklinik,Nein,Premium,Hoch,Nein,23
6,L00007,Herzklappen-Operation,Kardiologie,43175,10,Tagesklinik,Ja,Standard,Niedrig,Ja,25
7,L00008,Koronarangiographie,Kardiologie,34939,12,Stationaer,Nein,Luxus,Niedrig,Ja,2
8,L00009,Belastungs-EKG,Kardiologie,33435,5,Ambulant,Ja,Premium,Hoch,Ja,16
9,L00010,Langzeit-EKG,Kardiologie,8329,11,Stationaer,Nein,Luxus,Niedrig,Nein,1


8. List female patients / Weibliche Patienten auflisten

In [31]:
con.execute(
    """
    SELECT
        *
    FROM patients
    WHERE Geschlecht = 'W'
    """
).df()

,Patienten_ID,Vorname,Nachname,Herkunftsland,Alter_bei_Anmeldung,Geschlecht,Anmeldedatum,Sprache,Vorherige_Behandlung_Deutschland,Versicherungstyp,Familienanamnese,Familienstand,Bildungsgrad,Berufsstatus
0,P00001,Anna,Lebedev,Russland,13,W,2015-03-13,Russisch,Ja,Privat,Autoimmunerkrankungen,Ledig,Grundschule,Angestellt
1,P00003,Marina,Popov,Belarus,53,W,2015-06-22,Russisch,Ja,Privat,Autoimmunerkrankungen,Ledig,Abitur,Rentner
2,P00004,Vera,Kozlov,Ukraine,15,W,2015-07-13,Russisch,Ja,Ausland,Neurologische Erkrankungen,Verheiratet,Grundschule,Angestellt
3,P00008,Marina,Novikov,Lettland,60,W,2015-03-13,Russisch,Ja,Ausland,Autoimmunerkrankungen,Verwitwet,Abitur,Selbststaendig
4,P00009,Irina,Ivanov,Ukraine,27,W,2015-02-26,Russisch,Ja,Gesetzlich,Autoimmunerkrankungen,Ledig,Hochschulabschluss,Arbeitslos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6157,P12195,Valentina,Makarov,Russland,76,W,2024-03-08,Russisch,Nein,Selbstzahler,Onkologische Erkrankungen,Geschieden,Hochschulabschluss,Student
6158,P12196,Lyudmila,Kozlov,Russland,31,W,2024-09-25,Russisch,Nein,Gesetzlich,Herz-Kreislauf-Erkrankungen,Geschieden,Hochschulabschluss,Rentner
6159,P12197,Alla,Petrov,Kasachstan,46,W,2024-04-08,Russisch,Nein,Ausland,Herz-Kreislauf-Erkrankungen,Verwitwet,Mittlere Reife,Angestellt
6160,P12198,Galina,Lebedev,Russland,64,W,2024-09-25,Russisch,Nein,Ausland,Autoimmunerkrankungen,Verheiratet,Mittlere Reife,Angestellt


In [32]:
patients[patients["Geschlecht"] == "W"]

,Patienten_ID,Vorname,Nachname,Herkunftsland,Alter_bei_Anmeldung,Geschlecht,Anmeldedatum,Sprache,Vorherige_Behandlung_Deutschland,Versicherungstyp,Familienanamnese,Familienstand,Bildungsgrad,Berufsstatus
0,P00001,Anna,Lebedev,Russland,13,W,2015-03-13,Russisch,Ja,Privat,Autoimmunerkrankungen,Ledig,Grundschule,Angestellt
2,P00003,Marina,Popov,Belarus,53,W,2015-06-22,Russisch,Ja,Privat,Autoimmunerkrankungen,Ledig,Abitur,Rentner
3,P00004,Vera,Kozlov,Ukraine,15,W,2015-07-13,Russisch,Ja,Ausland,Neurologische Erkrankungen,Verheiratet,Grundschule,Angestellt
7,P00008,Marina,Novikov,Lettland,60,W,2015-03-13,Russisch,Ja,Ausland,Autoimmunerkrankungen,Verwitwet,Abitur,Selbststaendig
8,P00009,Irina,Ivanov,Ukraine,27,W,2015-02-26,Russisch,Ja,Gesetzlich,Autoimmunerkrankungen,Ledig,Hochschulabschluss,Arbeitslos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12194,P12195,Valentina,Makarov,Russland,76,W,2024-03-08,Russisch,Nein,Selbstzahler,Onkologische Erkrankungen,Geschieden,Hochschulabschluss,Student
12195,P12196,Lyudmila,Kozlov,Russland,31,W,2024-09-25,Russisch,Nein,Gesetzlich,Herz-Kreislauf-Erkrankungen,Geschieden,Hochschulabschluss,Rentner
12196,P12197,Alla,Petrov,Kasachstan,46,W,2024-04-08,Russisch,Nein,Ausland,Herz-Kreislauf-Erkrankungen,Verwitwet,Mittlere Reife,Angestellt
12197,P12198,Galina,Lebedev,Russland,64,W,2024-09-25,Russisch,Nein,Ausland,Autoimmunerkrankungen,Verheiratet,Mittlere Reife,Angestellt


In [33]:
services.columns

Index(['Leistungs_ID', 'Leistungsname', 'Kategorie', 'Kosten_EUR',
       'Dauer_Stunden', 'Behandlungsart', 'Nachsorge_erforderlich',
       'Qualitaetsstufe', 'Komplexitaet', 'Narkose_erforderlich',
       'Nachsorge_Tage'],
      dtype='object')

9. Show services costing more than 25000 EUR / Leistungen über 25000 EUR anzeigen

In [34]:
con.execute(
    """
    SELECT
        Leistungsname, Kosten_EUR
    FROM services
    WHERE Kosten_EUR > 25000
    """
).df()

,Leistungsname,Kosten_EUR
0,Echokardiographie,27089
1,Herzschrittmacher-Implantation,29066
2,Defibrillator-Implantation,36663
3,Herzklappen-Operation,43175
4,Koronarangiographie,34939
5,Belastungs-EKG,33435
6,Chemotherapie,39802
7,Immuntherapie,35321
8,Tumorentfernung,31195
9,Lymphknotenentfernung,35507


In [35]:
services[services["Kosten_EUR"] > 25000][["Leistungsname", "Kosten_EUR"]]

,Leistungsname,Kosten_EUR
0,Echokardiographie,27089
4,Herzschrittmacher-Implantation,29066
5,Defibrillator-Implantation,36663
6,Herzklappen-Operation,43175
7,Koronarangiographie,34939
8,Belastungs-EKG,33435
21,Chemotherapie,39802
23,Immuntherapie,35321
24,Tumorentfernung,31195
25,Lymphknotenentfernung,35507


10. Find active staff members / Aktive Mitarbeiter finden

In [36]:
con.execute(
    """
    SELECT
        Vorname, Nachname, Status
    FROM accompanying_staff
    WHERE Status = 'Aktiv'
    """
).df()

,Vorname,Nachname,Status
0,Anna,Mueller,Aktiv
1,Dmitri,Schmidt,Aktiv
2,Elena,Weber,Aktiv
3,Alexander,Wagner,Aktiv
4,Svetlana,Fischer,Aktiv
5,Pavel,Becker,Aktiv
6,Marina,Schulz,Aktiv
7,Viktor,Hoffmann,Aktiv
8,Irina,Schaefer,Aktiv
9,Sergei,Koch,Aktiv


In [37]:
accompanying_staff[accompanying_staff["Status"] == "Aktiv"][
    ["Vorname", "Nachname", "Status"]
]

,Vorname,Nachname,Status
0,Anna,Mueller,Aktiv
1,Dmitri,Schmidt,Aktiv
2,Elena,Weber,Aktiv
3,Alexander,Wagner,Aktiv
4,Svetlana,Fischer,Aktiv
5,Pavel,Becker,Aktiv
6,Marina,Schulz,Aktiv
7,Viktor,Hoffmann,Aktiv
8,Irina,Schaefer,Aktiv
9,Sergei,Koch,Aktiv


11. Show patients with private insurance / Patienten mit Privatversicherung anzeigen

In [38]:
patients[patients["Versicherungstyp"] == "Privat"][
    ["Vorname", "Nachname", "Versicherungstyp"]
]

,Vorname,Nachname,Versicherungstyp
0,Anna,Lebedev,Privat
2,Marina,Popov,Privat
11,Kirill,Sokolov,Privat
14,Artem,Andreev,Privat
18,Vladimir,Stepanov,Privat
...,...,...,...
12174,Elena,Makarov,Privat
12176,Yuri,Makarov,Privat
12180,Larisa,Mikhailov,Privat
12185,Dmitri,Stepanov,Privat


12. List treatments in Berlin / Behandlungen in Berlin auflisten

In [39]:
treatment[treatment["Behandlungsort"] == "Berlin"]

,Behandlungs_ID,Patienten_ID,Leistungs_ID,Mitarbeiter_ID,Anfragedatum,Beratungsdatum,Behandlungsdatum,Abschlussdatum,Status,Behandlungsort,...,Dolmetscherstunden,Leistungskosten_EUR,Dolmetscherkosten_EUR,Transportkosten_EUR,Unterkunftskosten_EUR,Zusatzkosten_EUR,Gesamtkosten_EUR,Komplikationen,Patientenzufriedenheit,Nachsorge_erforderlich
16,B00017,P00418,L00058,M005,2015-10-04,2015-10-11,2015-11-26,2015-12-03,Storniert,Berlin,...,18,30881,1484,683,643,646,34337,Nein,Zufrieden,Teilweise
24,B00025,P00307,L00066,M003,2015-01-13,2015-01-14,2015-01-24,2015-01-29,Beratung_durchgefuehrt,Berlin,...,16,11690,1273,788,436,1114,15301,Nein,Unzufrieden,Teilweise
36,B00037,P00436,L00038,M005,2015-01-21,2015-01-28,2015-02-04,2015-02-27,Storniert,Berlin,...,11,10500,1224,195,847,503,13269,Nein,Zufrieden,Ja
39,B00040,P00597,L00040,M010,2015-03-23,2015-03-30,2015-05-23,2015-06-08,Anfrage,Berlin,...,19,15669,1362,555,2957,506,21049,Nein,Sehr_zufrieden,Teilweise
71,B00072,P00606,L00042,M008,2015-03-28,2015-04-08,2015-05-12,2015-06-07,Abgeschlossen,Berlin,...,12,21359,759,81,1686,773,24658,Nein,Unzufrieden,Teilweise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8660,B08661,P08742,L00005,M010,2024-08-07,2024-08-12,2024-09-18,2024-09-25,Termin_vereinbart,Berlin,...,2,36746,556,180,2735,1429,41646,Nein,Unzufrieden,Ja
8706,B08707,P08150,L00064,M010,2024-10-10,2024-10-17,2024-11-01,2024-11-24,Angebot_erstellt,Berlin,...,14,2927,214,306,696,236,4379,Ja,Unzufrieden,Nein
8719,B08720,P08458,L00013,M009,2024-11-14,2024-11-27,2024-12-26,2025-01-12,Behandlung_abgeschlossen,Berlin,...,16,15993,1460,413,701,1515,20082,Nein,Zufrieden,Nein
8726,B08727,P08590,L00069,M005,2024-09-07,2024-09-17,2024-11-01,2024-11-16,Angebot_erstellt,Berlin,...,16,17836,1028,204,2641,417,22126,Nein,Zufrieden,Ja


13. Show completed treatments / Abgeschlossene Behandlungen anzeigen

In [40]:
treatment[treatment["Status"] == "Abgeschlossen"]

,Behandlungs_ID,Patienten_ID,Leistungs_ID,Mitarbeiter_ID,Anfragedatum,Beratungsdatum,Behandlungsdatum,Abschlussdatum,Status,Behandlungsort,...,Dolmetscherstunden,Leistungskosten_EUR,Dolmetscherkosten_EUR,Transportkosten_EUR,Unterkunftskosten_EUR,Zusatzkosten_EUR,Gesamtkosten_EUR,Komplikationen,Patientenzufriedenheit,Nachsorge_erforderlich
1,B00002,P00498,L00079,M006,2015-12-23,2016-01-05,2016-02-04,2016-02-24,Abgeschlossen,Hamburg,...,3,5573,383,55,2290,1372,9673,Ja,Unzufrieden,Ja
11,B00012,P00230,L00022,M002,2015-07-18,2015-07-26,2015-09-04,2015-09-12,Abgeschlossen,Duisburg,...,8,8826,655,716,2891,445,13533,Ja,Sehr_zufrieden,Ja
14,B00015,P00005,L00009,M005,2015-03-20,2015-03-31,2015-05-21,2015-05-22,Abgeschlossen,Duesseldorf,...,14,44296,499,666,707,1376,47544,Ja,Zufrieden,Nein
40,B00041,P00076,L00056,M002,2015-12-23,2015-12-26,2016-02-24,2016-03-17,Abgeschlossen,Duesseldorf,...,5,22724,683,231,2526,1641,27805,Ja,Zufrieden,Teilweise
49,B00050,P00444,L00071,M002,2015-05-20,2015-05-28,2015-06-12,2015-06-21,Abgeschlossen,Nuremberg,...,8,36657,1212,518,2217,909,41513,Nein,Zufrieden,Teilweise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8715,B08716,P08598,L00006,M009,2024-06-17,2024-07-01,2024-07-24,2024-07-27,Abgeschlossen,Duesseldorf,...,19,31519,1327,506,701,122,34175,Nein,Unzufrieden,Ja
8732,B08733,P08602,L00016,M009,2024-08-01,2024-08-12,2024-09-08,2024-09-27,Abgeschlossen,Cologne,...,9,29127,608,764,979,23,31501,Nein,Sehr_zufrieden,Nein
8735,B08736,P08395,L00071,M008,2024-12-07,2024-12-13,2025-01-26,2025-02-22,Abgeschlossen,Nuremberg,...,2,14968,819,600,1380,910,18677,Ja,Sehr_zufrieden,Teilweise
8737,B08738,P08090,L00024,M004,2024-06-23,2024-07-03,2024-08-24,2024-09-17,Abgeschlossen,Dortmund,...,11,34112,492,454,1371,177,36606,Ja,Sehr_zufrieden,Nein


14. Find accommodations in Munich / Unterkünfte in München finden

In [41]:
accommodations[accommodations["Stadt"] == "Munich"]

,Unterkunft_ID,Name,Stadt,Typ,Sterne,Preis_pro_Nacht_EUR,Medizinische_Betreuung,Russischsprachiges_Personal,Transfer_Service,Kuechenzeile_verfuegbar,Anzahl_Zimmer
1,U002,Kempinski Munich,Munich,Hotel,5,160,Ja,Ja,Nein,Nein,26


15. Show 5-star hotels / 5-Sterne-Hotels anzeigen

In [42]:
accommodations[accommodations["Sterne"] == 5]

,Unterkunft_ID,Name,Stadt,Typ,Sterne,Preis_pro_Nacht_EUR,Medizinische_Betreuung,Russischsprachiges_Personal,Transfer_Service,Kuechenzeile_verfuegbar,Anzahl_Zimmer
0,U001,Hotel Adlon Berlin,Berlin,Hotel,5,159,Ja,Ja,Nein,Ja,50
1,U002,Kempinski Munich,Munich,Hotel,5,160,Ja,Ja,Nein,Nein,26


16. List patients by registration date / Patienten nach Anmeldedatum sortieren

In [43]:
patients.reset_index().sort_values("Anmeldedatum", ascending=True)

,index,Patienten_ID,Vorname,Nachname,Herkunftsland,Alter_bei_Anmeldung,Geschlecht,Anmeldedatum,Sprache,Vorherige_Behandlung_Deutschland,Versicherungstyp,Familienanamnese,Familienstand,Bildungsgrad,Berufsstatus
277,277,P00278,Maria,Kozlov,Ukraine,18,W,2015-01-01,Russisch,Ja,Selbstzahler,Erbliche Stoffwechselerkrankungen,Verwitwet,Mittlere Reife,Arbeitslos
671,671,P00672,Vera,Stepanov,Litauen,12,W,2015-01-02,Russisch,Nein,Privat,Neurologische Erkrankungen,Verwitwet,Hochschulabschluss,Rentner
315,315,P00316,Ekaterina,Lebedev,Ukraine,36,W,2015-01-02,Russisch,Ja,Privat,Diabetes mellitus,Verwitwet,Mittlere Reife,Angestellt
75,75,P00076,Larisa,Novikov,Belarus,63,W,2015-01-02,Russisch,Nein,Ausland,Erbliche Stoffwechselerkrankungen,Verheiratet,Hochschulabschluss,Student
643,643,P00644,Yulia,Makarov,Russland,25,W,2015-01-03,Russisch,Nein,Ausland,Bluthochdruck,Verheiratet,Abitur,Rentner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12107,12107,P12108,Anastasia,Kozlov,Russland,76,W,2024-12-28,Russisch,Nein,Gesetzlich,Autoimmunerkrankungen,Verwitwet,Grundschule,Student
11293,11293,P11294,Kirill,Kozlov,Belarus,57,M,2024-12-29,Russisch,Nein,Selbstzahler,Erbliche Stoffwechselerkrankungen,Verwitwet,Hochschulabschluss,Angestellt
11763,11763,P11764,Tatiana,Volkov,Russland,27,W,2024-12-29,Russisch,Nein,Privat,Bluthochdruck,Geschieden,Hochschulabschluss,Student
11502,11502,P11503,Larisa,Lebedev,Ukraine,16,W,2024-12-30,Russisch,Nein,Selbstzahler,Psychische Erkrankungen,Ledig,Mittlere Reife,Arbeitslos


17. Show most expensive services / Teuerste Leistungen anzeigen

In [44]:
services.reset_index().sort_values("Kosten_EUR", ascending=False)

,index,Leistungs_ID,Leistungsname,Kategorie,Kosten_EUR,Dauer_Stunden,Behandlungsart,Nachsorge_erforderlich,Qualitaetsstufe,Komplexitaet,Narkose_erforderlich,Nachsorge_Tage
29,29,L00030,Palliativbehandlung,Onkologie,44148,7,Ambulant,Ja,Premium,Niedrig,Ja,27
27,27,L00028,Targeted Therapy,Onkologie,43556,8,Ambulant,Ja,Premium,Hoch,Ja,22
6,6,L00007,Herzklappen-Operation,Kardiologie,43175,10,Tagesklinik,Ja,Standard,Niedrig,Ja,25
21,21,L00022,Chemotherapie,Onkologie,39802,10,Stationaer,Nein,Standard,Mittel,Nein,14
26,26,L00027,Knochenmarktransplantation,Onkologie,37249,3,Tagesklinik,Nein,Luxus,Mittel,Nein,4
...,...,...,...,...,...,...,...,...,...,...,...,...
69,69,L00070,Szintigraphie,Radiologie,151,2,Tagesklinik,Ja,Premium,Mittel,Ja,22
52,52,L00053,Nierenwerte,Labordiagnostik,132,3,Ambulant,Nein,Standard,Niedrig,Ja,9
50,50,L00051,Blutbild komplett,Labordiagnostik,121,1,Stationaer,Ja,Premium,Mittel,Ja,7
60,60,L00061,MRT Wirbelsaeule,Radiologie,97,1,Ambulant,Nein,Luxus,Hoch,Nein,8


18. Find youngest patients / Jüngste Patienten finden

In [45]:
youngest = patients.reset_index().sort_values(
    ["Alter_bei_Anmeldung", "Anmeldedatum"], ascending=True
)
youngest.head(10)

,index,Patienten_ID,Vorname,Nachname,Herkunftsland,Alter_bei_Anmeldung,Geschlecht,Anmeldedatum,Sprache,Vorherige_Behandlung_Deutschland,Versicherungstyp,Familienanamnese,Familienstand,Bildungsgrad,Berufsstatus
28,28,P00029,Denis,Ivanov,Ukraine,3,M,2015-07-09,Russisch,Ja,Selbstzahler,Onkologische Erkrankungen,Ledig,Abitur,Rentner
1045,1045,P01046,Larisa,Yakovlev,Ukraine,3,W,2016-07-16,Russisch,Ja,Selbstzahler,Keine bekannten Erkrankungen,Ledig,Grundschule,Arbeitslos
1235,1235,P01236,Pavel,Stepanov,Russland,3,M,2016-12-03,Russisch,Nein,Ausland,Erbliche Stoffwechselerkrankungen,Ledig,Mittlere Reife,Student
857,857,P00858,Alexander,Nikitin,Ukraine,3,M,2016-12-28,Russisch,Ja,Selbstzahler,Herz-Kreislauf-Erkrankungen,Verheiratet,Hochschulabschluss,Rentner
2494,2494,P02495,Oleg,Ivanov,Ukraine,3,M,2017-03-12,Russisch,Nein,Ausland,Neurologische Erkrankungen,Geschieden,Grundschule,Angestellt
2803,2803,P02804,Yuri,Mikhailov,Moldawien,3,M,2017-07-30,Russisch,Nein,Selbstzahler,Herz-Kreislauf-Erkrankungen,Ledig,Hochschulabschluss,Arbeitslos
3016,3016,P03017,Sergei,Petrov,Belarus,3,M,2018-05-11,Russisch,Ja,Selbstzahler,Onkologische Erkrankungen,Geschieden,Abitur,Rentner
5379,5379,P05380,Tatiana,Andreev,Russland,3,W,2019-05-12,Russisch,Nein,Privat,Bluthochdruck,Geschieden,Abitur,Student
4131,4131,P04132,Pavel,Sidorov,Russland,3,M,2019-05-25,Russisch,Nein,Ausland,Keine bekannten Erkrankungen,Verheiratet,Grundschule,Rentner
4285,4285,P04286,Olga,Ivanov,Ukraine,3,W,2019-09-21,Russisch,Ja,Privat,Onkologische Erkrankungen,Geschieden,Grundschule,Arbeitslos


19. Show staff with highest hourly rate / Mitarbeiter mit höchstem Stundenlohn anzeigen

In [46]:
highest_hour_rate = accompanying_staff.reset_index().sort_values(
    "Stundenlohn_EUR", ascending=False
)
highest_hour_rate[["Vorname", "Nachname", "Stundenlohn_EUR"]].head(10)

,Vorname,Nachname,Stundenlohn_EUR
0,Anna,Mueller,53
2,Elena,Weber,50
5,Pavel,Becker,50
3,Alexander,Wagner,48
4,Svetlana,Fischer,46
9,Sergei,Koch,45
8,Irina,Schaefer,31
6,Marina,Schulz,28
7,Viktor,Hoffmann,28
1,Dmitri,Schmidt,25


20. List treatments by total cost / Behandlungen nach Gesamtkosten sortieren

In [47]:
total_cost = treatment.reset_index().sort_values("Gesamtkosten_EUR", ascending=False)
total_cost.head(10)

,index,Behandlungs_ID,Patienten_ID,Leistungs_ID,Mitarbeiter_ID,Anfragedatum,Beratungsdatum,Behandlungsdatum,Abschlussdatum,Status,...,Dolmetscherstunden,Leistungskosten_EUR,Dolmetscherkosten_EUR,Transportkosten_EUR,Unterkunftskosten_EUR,Zusatzkosten_EUR,Gesamtkosten_EUR,Komplikationen,Patientenzufriedenheit,Nachsorge_erforderlich
2888,2888,B02889,P02859,L00048,M009,2018-09-27,2018-09-28,2018-10-18,2018-11-14,Anreise_geplant,...,7,44934,1273,512,2690,1967,51376,Ja,Neutral,Ja
4545,4545,B04546,P05033,L00041,M002,2020-11-26,2020-12-06,2020-12-19,2021-01-16,Behandlung_abgeschlossen,...,20,44828,1451,415,2634,1873,51201,Nein,Unzufrieden,Teilweise
1558,1558,B01559,P01863,L00063,M009,2017-11-01,2017-11-10,2017-12-21,2018-01-07,Beratung_durchgefuehrt,...,20,44206,1389,779,2712,1847,50933,Nein,Neutral,Nein
3350,3350,B03351,P03211,L00036,M005,2019-03-25,2019-04-04,2019-04-14,2019-05-01,Termin_vereinbart,...,19,44619,1491,765,2557,1471,50903,Nein,Neutral,Ja
5138,5138,B05139,P05626,L00036,M007,2021-04-24,2021-05-04,2021-05-18,2021-06-17,Termin_vereinbart,...,20,44570,1181,721,2696,1634,50802,Nein,Sehr_zufrieden,Nein
6411,6411,B06412,P06804,L00058,M007,2022-03-27,2022-04-01,2022-05-29,2022-06-25,Anfrage,...,7,44711,1228,270,2654,1892,50755,Nein,Sehr_zufrieden,Ja
4904,4904,B04905,P04923,L00068,M008,2020-11-08,2020-11-20,2021-01-07,2021-01-27,Anfrage,...,14,44457,1447,303,2861,1510,50578,Nein,Unzufrieden,Teilweise
4763,4763,B04764,P04318,L00025,M010,2020-08-03,2020-08-15,2020-08-30,2020-09-17,Beratung_geplant,...,10,44747,823,702,2697,1417,50386,Nein,Neutral,Ja
7931,7931,B07932,P07930,L00045,M001,2023-09-20,2023-10-04,2023-10-11,2023-10-15,Nachsorge,...,19,44591,1286,484,2723,1289,50373,Ja,Unzufrieden,Ja
723,723,B00724,P00759,L00054,M009,2016-02-06,2016-02-11,2016-02-26,2016-03-25,Beratung_geplant,...,4,44694,658,383,2861,1770,50366,Ja,Neutral,Nein


21. Count patients by country / Patienten nach Land zählen

In [48]:
patients.groupby("Herkunftsland").size().sort_values(ascending=False)

Herkunftsland
Russland      5566
Ukraine       3060
Belarus        925
Kasachstan     839
Lettland       495
Estland        386
Litauen        351
Moldawien      247
Georgien       225
Armenien       106
dtype: int64

22. Average age of patients / Durchschnittsalter der Patienten

In [49]:
round(patients["Alter_bei_Anmeldung"].mean(), 2)

47.5

23. Total cost of all services / Gesamtkosten aller Leistungen

In [50]:
services["Kosten_EUR"].sum()

1086493

24. Count treatments by status / Behandlungen nach Status zählen

In [51]:
treatment["Status"].value_counts().reset_index(name="Counts by status")

,Status,Counts by status
0,Termin_vereinbart,776
1,Nachsorge,775
2,Behandlung_abgeschlossen,756
3,Storniert,756
4,Angebot_angenommen,752
5,Abgeschlossen,736
6,Anreise_geplant,720
7,Anfrage,714
8,Angebot_erstellt,712
9,In_Behandlung,700


In [52]:
treatment.groupby("Status").size().sort_values(ascending=False)

Status
Termin_vereinbart           776
Nachsorge                   775
Behandlung_abgeschlossen    756
Storniert                   756
Angebot_angenommen          752
Abgeschlossen               736
Anreise_geplant             720
Anfrage                     714
Angebot_erstellt            712
In_Behandlung               700
Beratung_durchgefuehrt      685
Beratung_geplant            668
dtype: int64

25. Maximum service cost / Maximale Leistungskosten

In [53]:
services["Kosten_EUR"].max()

44148

26. Minimum patient age / Minimales Patientenalter

In [54]:
patients["Alter_bei_Anmeldung"].min()

3

27. Count services by category / Leistungen nach Kategorie zählen

In [55]:
services.groupby("Kategorie").size().sort_values(ascending=False)

Kategorie
Orthopaedie          11
Radiologie           11
Kardiologie          10
Labordiagnostik      10
Neurologie           10
Onkologie            10
Augenheilkunde        9
Gastroenterologie     9
dtype: int64

 28. Average treatment duration / Durchschnittliche Behandlungsdauer

In [56]:
round(services["Dauer_Stunden"].mean(), 2)

4.58

29. Count patients by gender / Patienten nach Geschlecht zählen

In [57]:
patients.groupby("Geschlecht").size().sort_values(ascending=False)

Geschlecht
W    6162
M    6038
dtype: int64

30. Show unique countries / Eindeutige Länder anzeigen

In [58]:
for land in patients["Herkunftsland"].unique():
    print(land)

Russland
Moldawien
Belarus
Ukraine
Lettland
Kasachstan
Estland
Litauen
Georgien
Armenien


31. List unique treatment locations / Eindeutige Behandlungsorte auflisten

In [59]:
for ort in treatment["Behandlungsort"].unique():
    print(ort)

Duisburg
Hamburg
Nuremberg
Munich
Hannover
Dortmund
Frankfurt
Cologne
Dresden
Duesseldorf
Berlin
Essen
Stuttgart
Leipzig
Bremen


32. Show unique service categories / Eindeutige Leistungskategorien anzeigen

In [60]:
for serv in services["Kategorie"].unique():
    print(serv)

Kardiologie
Orthopaedie
Onkologie
Neurologie
Augenheilkunde
Labordiagnostik
Radiologie
Gastroenterologie


33. Count accommodations by type / Unterkünfte nach Typ zählen

In [61]:
accommodations.groupby("Typ").size().sort_values(ascending=False)

Typ
Hotel                      5
Apartment                  1
Klinikhotel                1
Medizinische_Unterkunft    1
Pension                    1
Rehabilitationszentrum     1
dtype: int64

34. Average accommodation price / Durchschnittlicher Unterkunftspreis

In [62]:
accommodations["Preis_pro_Nacht_EUR"].mean()

195.9

35. Show patients with university education / Patienten mit Hochschulbildung anzeigen

In [63]:
patients[patients["Bildungsgrad"] == "Hochschulabschluss"][
    ["Vorname", "Nachname", "Bildungsgrad"]
]

,Vorname,Nachname,Bildungsgrad
8,Irina,Ivanov,Hochschulabschluss
15,Roman,Lebedev,Hochschulabschluss
16,Sergei,Kozlov,Hochschulabschluss
21,Dmitri,Orlov,Hochschulabschluss
27,Natasha,Petrov,Hochschulabschluss
...,...,...,...
12189,Pavel,Petrov,Hochschulabschluss
12191,Roman,Novikov,Hochschulabschluss
12194,Valentina,Makarov,Hochschulabschluss
12195,Lyudmila,Kozlov,Hochschulabschluss


36. Find treatments requiring anesthesia / Behandlungen mit Narkose finden

In [64]:
services[services["Narkose_erforderlich"] == "Ja"][
    ["Leistungsname", "Narkose_erforderlich"]
]

,Leistungsname,Narkose_erforderlich
0,Echokardiographie,Ja
1,Herzkatheteruntersuchung,Ja
2,Bypass-Operation,Ja
3,Stent-Implantation,Ja
6,Herzklappen-Operation,Ja
7,Koronarangiographie,Ja
8,Belastungs-EKG,Ja
10,Hueftgelenk-Endoprothese,Ja
12,Wirbelsaeulen-Operation,Ja
13,Arthroskopie Knie,Ja


37. Show married patients / Verheiratete Patienten anzeigen

In [65]:
patients[patients["Familienstand"] == "Verheiratet"][
    ["Vorname", "Nachname", "Familienstand"]
]

,Vorname,Nachname,Familienstand
3,Vera,Kozlov,Verheiratet
4,Igor,Kozlov,Verheiratet
6,Igor,Ivanov,Verheiratet
10,Larisa,Smirnov,Verheiratet
12,Vladimir,Yakovlev,Verheiratet
...,...,...,...
12184,Alexei,Mikhailov,Verheiratet
12191,Roman,Novikov,Verheiratet
12197,Galina,Lebedev,Verheiratet
12198,Galina,Morozov,Verheiratet


38. List premium quality services / Premium-Qualitätsleistungen auflisten


In [66]:
services[services["Qualitaetsstufe"] == "Premium"]

,Leistungs_ID,Leistungsname,Kategorie,Kosten_EUR,Dauer_Stunden,Behandlungsart,Nachsorge_erforderlich,Qualitaetsstufe,Komplexitaet,Narkose_erforderlich,Nachsorge_Tage
0,L00001,Echokardiographie,Kardiologie,27089,4,Tagesklinik,Nein,Premium,Hoch,Ja,15
1,L00002,Herzkatheteruntersuchung,Kardiologie,20740,6,Ambulant,Nein,Premium,Niedrig,Ja,4
3,L00004,Stent-Implantation,Kardiologie,21151,12,Ambulant,Nein,Premium,Niedrig,Ja,3
4,L00005,Herzschrittmacher-Implantation,Kardiologie,29066,9,Stationaer,Ja,Premium,Hoch,Nein,29
5,L00006,Defibrillator-Implantation,Kardiologie,36663,12,Tagesklinik,Nein,Premium,Hoch,Nein,23
8,L00009,Belastungs-EKG,Kardiologie,33435,5,Ambulant,Ja,Premium,Hoch,Ja,16
10,L00011,Hueftgelenk-Endoprothese,Orthopaedie,24878,1,Ambulant,Ja,Premium,Mittel,Ja,21
14,L00015,Arthroskopie Schulter,Orthopaedie,17827,4,Stationaer,Ja,Premium,Hoch,Ja,15
18,L00019,Schulter-Endoprothese,Orthopaedie,24260,5,Tagesklinik,Nein,Premium,Niedrig,Ja,29
27,L00028,Targeted Therapy,Onkologie,43556,8,Ambulant,Ja,Premium,Hoch,Ja,22


39. Show treatments with complications / Behandlungen mit Komplikationen anzeigen

In [67]:
treatment[treatment["Komplikationen"] == "Ja"]

,Behandlungs_ID,Patienten_ID,Leistungs_ID,Mitarbeiter_ID,Anfragedatum,Beratungsdatum,Behandlungsdatum,Abschlussdatum,Status,Behandlungsort,...,Dolmetscherstunden,Leistungskosten_EUR,Dolmetscherkosten_EUR,Transportkosten_EUR,Unterkunftskosten_EUR,Zusatzkosten_EUR,Gesamtkosten_EUR,Komplikationen,Patientenzufriedenheit,Nachsorge_erforderlich
1,B00002,P00498,L00079,M006,2015-12-23,2016-01-05,2016-02-04,2016-02-24,Abgeschlossen,Hamburg,...,3,5573,383,55,2290,1372,9673,Ja,Unzufrieden,Ja
5,B00006,P00085,L00035,M004,2015-01-25,2015-02-02,2015-03-19,2015-04-04,Behandlung_abgeschlossen,Dortmund,...,17,16651,1255,655,2798,247,21606,Ja,Sehr_zufrieden,Ja
10,B00011,P00286,L00006,M001,2015-01-14,2015-01-20,2015-01-31,2015-02-01,Angebot_angenommen,Cologne,...,16,20165,1074,745,778,364,23126,Ja,Unzufrieden,Nein
11,B00012,P00230,L00022,M002,2015-07-18,2015-07-26,2015-09-04,2015-09-12,Abgeschlossen,Duisburg,...,8,8826,655,716,2891,445,13533,Ja,Sehr_zufrieden,Ja
13,B00014,P00620,L00035,M003,2015-10-25,2015-11-04,2015-12-21,2016-01-15,Beratung_geplant,Dresden,...,15,23632,1172,596,654,223,26277,Ja,Unzufrieden,Nein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8745,B08746,P08486,L00014,M007,2024-02-04,2024-02-11,2024-03-17,2024-04-01,Angebot_angenommen,Cologne,...,10,26134,682,718,2785,272,30591,Ja,Zufrieden,Ja
8746,B08747,P08700,L00063,M009,2024-05-30,2024-06-02,2024-06-29,2024-07-25,Termin_vereinbart,Hamburg,...,2,30808,483,791,434,428,32944,Ja,Neutral,Ja
8747,B08748,P08450,L00037,M001,2024-10-21,2024-10-28,2024-11-07,2024-12-07,Anreise_geplant,Duisburg,...,17,27922,778,773,518,25,30016,Ja,Unzufrieden,Ja
8748,B08749,P08655,L00068,M009,2024-09-30,2024-10-09,2024-11-04,2024-11-30,Anreise_geplant,Frankfurt,...,9,5724,1031,780,1874,871,10280,Ja,Sehr_zufrieden,Teilweise


 40. Find satisfied patients / Zufriedene Patienten finden

In [68]:
treatment[treatment["Patientenzufriedenheit"] == "Zufrieden"]

,Behandlungs_ID,Patienten_ID,Leistungs_ID,Mitarbeiter_ID,Anfragedatum,Beratungsdatum,Behandlungsdatum,Abschlussdatum,Status,Behandlungsort,...,Dolmetscherstunden,Leistungskosten_EUR,Dolmetscherkosten_EUR,Transportkosten_EUR,Unterkunftskosten_EUR,Zusatzkosten_EUR,Gesamtkosten_EUR,Komplikationen,Patientenzufriedenheit,Nachsorge_erforderlich
2,B00003,P00494,L00046,M004,2015-08-19,2015-08-29,2015-10-04,2015-11-01,Anreise_geplant,Nuremberg,...,2,21937,1272,684,1988,197,26078,Nein,Zufrieden,Teilweise
3,B00004,P00298,L00053,M003,2015-05-18,2015-05-29,2015-07-13,2015-08-01,Nachsorge,Munich,...,19,44871,544,148,2326,927,48816,Nein,Zufrieden,Ja
8,B00009,P00033,L00020,M009,2015-11-27,2015-12-07,2015-12-15,2016-01-11,Nachsorge,Dortmund,...,19,31944,465,261,1375,672,34717,Nein,Zufrieden,Ja
14,B00015,P00005,L00009,M005,2015-03-20,2015-03-31,2015-05-21,2015-05-22,Abgeschlossen,Duesseldorf,...,14,44296,499,666,707,1376,47544,Ja,Zufrieden,Nein
16,B00017,P00418,L00058,M005,2015-10-04,2015-10-11,2015-11-26,2015-12-03,Storniert,Berlin,...,18,30881,1484,683,643,646,34337,Nein,Zufrieden,Teilweise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8722,B08723,P08324,L00007,M004,2024-08-10,2024-08-22,2024-09-10,2024-09-29,Anreise_geplant,Munich,...,19,3520,1432,395,1222,382,6951,Ja,Zufrieden,Ja
8726,B08727,P08590,L00069,M005,2024-09-07,2024-09-17,2024-11-01,2024-11-16,Angebot_erstellt,Berlin,...,16,17836,1028,204,2641,417,22126,Nein,Zufrieden,Ja
8739,B08740,P08371,L00001,M008,2024-10-20,2024-10-28,2024-12-10,2025-01-02,Abgeschlossen,Munich,...,9,28724,335,369,2594,1815,33837,Ja,Zufrieden,Nein
8743,B08744,P08698,L00076,M002,2024-04-20,2024-05-01,2024-06-04,2024-06-27,Angebot_erstellt,Bremen,...,5,20115,1102,324,2739,1530,25810,Ja,Zufrieden,Teilweise


## Intermediate Level | Part 1

1. Join patients with their treatments / Patienten mit ihren Behandlungen verknüpfen

In [69]:
patients_treatments = pd.merge(patients, treatment, on="Patienten_ID", how="inner")
patients_treatments[["Vorname", "Nachname", "Behandlungsdatum", "Status"]]

,Vorname,Nachname,Behandlungsdatum,Status
0,Anna,Lebedev,2015-08-22,Anreise_geplant
1,Marina,Popov,2015-07-03,Behandlung_abgeschlossen
2,Vera,Kozlov,2016-02-19,Angebot_erstellt
3,Vera,Kozlov,2015-07-02,Storniert
4,Igor,Kozlov,2015-05-21,Abgeschlossen
...,...,...,...,...
8745,Oksana,Stepanov,2024-10-09,Abgeschlossen
8746,Alexei,Ivanov,2024-09-23,Abgeschlossen
8747,Alexei,Ivanov,2024-03-22,Behandlung_abgeschlossen
8748,Alexei,Ivanov,2024-02-21,Angebot_angenommen


2. Show treatments with service details / Behandlungen mit Leistungsdetails anzeigen

In [70]:
result = treatment.merge(
    patients[["Patienten_ID", "Nachname"]], on="Patienten_ID"
).merge(services[["Leistungs_ID", "Leistungsname", "Kategorie"]], on="Leistungs_ID")[
    ["Behandlungs_ID", "Nachname", "Leistungsname", "Kategorie", "Gesamtkosten_EUR"]
]

result

,Behandlungs_ID,Nachname,Leistungsname,Kategorie,Gesamtkosten_EUR
0,B00001,Nikitin,Roentgen Thorax,Radiologie,39125
1,B00002,Fedorov,Reflux-Operation,Gastroenterologie,9673
2,B00003,Ivanov,Hornhauttransplantation,Augenheilkunde,26078
3,B00004,Yakovlev,Nierenwerte,Labordiagnostik,48816
4,B00005,Sokolov,Koloskopie,Gastroenterologie,42858
...,...,...,...,...,...
8745,B08746,Morozov,Arthroskopie Knie,Orthopaedie,30591
8746,B08747,Popov,CT Thorax,Radiologie,32944
8747,B08748,Mikhailov,Epilepsie-Behandlung,Neurologie,30016
8748,B08749,Makarov,Angiographie,Radiologie,10280


3. Count treatments per patient / Behandlungen pro Patient zählen

In [71]:
treatments_count = (
    patients.merge(treatment, on="Patienten_ID", how="left")
    .groupby(["Patienten_ID", "Vorname", "Nachname"], as_index=False)
    .agg(treatments_count=("Behandlungs_ID", "count"))
)

treatments_count.sort_values("treatments_count", ascending=False)

,Patienten_ID,Vorname,Nachname,treatments_count
5280,P05281,Ekaterina,Nikitin,6
7050,P07051,Sergei,Morozov,6
5578,P05579,Boris,Popov,6
1579,P01580,Marina,Mikhailov,5
3606,P03607,Marina,Lebedev,5
...,...,...,...,...
7323,P07324,Vera,Andreev,0
1238,P01239,Valentina,Yakovlev,0
1242,P01243,Dmitri,Fedorov,0
1244,P01245,Maria,Novikov,0


4. Average treatment cost by country / Durchschnittliche Behandlungskosten nach Land

In [72]:
avg_costs = (
    patients.merge(treatment, on="Patienten_ID", how="inner")
    .groupby("Herkunftsland", as_index=False)
    .agg(avg_costs=("Gesamtkosten_EUR", "mean"))
)

avg_costs.round(2).sort_values("avg_costs", ascending=False)

,Herkunftsland,avg_costs
2,Estland,27524.42
0,Armenien,27235.41
1,Belarus,26935.34
6,Litauen,26813.20
3,Georgien,26779.44
7,Moldawien,26735.91
9,Ukraine,26487.98
8,Russland,26325.73
4,Kasachstan,26242.61
5,Lettland,26165.49


5. Staff workload analysis / Arbeitsbelastung des Personals analysieren

In [73]:
workload = (
    accompanying_staff.merge(treatment, on="Mitarbeiter_ID", how="left")
    .groupby(["Mitarbeiter_ID", "Vorname", "Nachname"], as_index=False)
    .agg(staff_workload=("Behandlungs_ID", "count"))
)

workload.sort_values("staff_workload", ascending=False)

,Mitarbeiter_ID,Vorname,Nachname,staff_workload
2,M003,Elena,Weber,947
0,M001,Anna,Mueller,895
8,M009,Irina,Schaefer,882
3,M004,Alexander,Wagner,875
1,M002,Dmitri,Schmidt,866
6,M007,Marina,Schulz,866
4,M005,Svetlana,Fischer,861
9,M010,Sergei,Koch,860
5,M006,Pavel,Becker,852
7,M008,Viktor,Hoffmann,846


6. Most expensive treatments by category / Teuerste Behandlungen nach Kategorie

In [74]:
max_costs = (
    services.merge(treatment, on="Leistungs_ID", how="inner")
    .groupby("Kategorie", as_index=False)
    .agg(max_costs=("Gesamtkosten_EUR", "max"))
)

max_costs.sort_values("max_costs", ascending=False)

,Kategorie,max_costs
0,Augenheilkunde,51376
4,Neurologie,51201
7,Radiologie,50933
3,Labordiagnostik,50755
5,Onkologie,50386
6,Orthopaedie,50191
1,Gastroenterologie,50187
2,Kardiologie,50181


7. Patients with multiple treatments / Patienten mit mehreren Behandlungen

In [75]:
multi_treatments = (
    patients.merge(treatment, on="Patienten_ID", how="inner")
    .groupby(["Patienten_ID", "Vorname", "Nachname"], as_index=False)
    .agg(treatments_count=("Behandlungs_ID", "count"))
)

multi_treatments = multi_treatments[multi_treatments["treatments_count"] > 1]
multi_treatments.sort_values("treatments_count", ascending=False)

,Patienten_ID,Vorname,Nachname,treatments_count
3338,P05281,Ekaterina,Nikitin,6
3525,P05579,Boris,Popov,6
4466,P07051,Sergei,Morozov,6
4345,P06875,Irina,Orlov,5
4574,P07225,Viktor,Nikitin,5
...,...,...,...,...
2085,P03351,Oksana,Novikov,2
2084,P03349,Natasha,Morozov,2
2083,P03346,Yuri,Smirnov,2
2073,P03330,Irina,Stepanov,2


8. Revenue by treatment location / Umsatz nach Behandlungsort

In [76]:
revenue_by_location = (
    treatment.groupby("Behandlungsort", as_index=False)
    .agg(total_revenue=("Gesamtkosten_EUR", "sum"))
    .sort_values("total_revenue", ascending=False)
)

revenue_by_location

,Behandlungsort,total_revenue
12,Munich,16373804
6,Duisburg,16111217
5,Duesseldorf,15910533
7,Essen,15891876
14,Stuttgart,15820808
10,Hannover,15782548
0,Berlin,15482557
13,Nuremberg,15355004
11,Leipzig,15298037
1,Bremen,15261261


9. Treatments requiring follow-up care / Behandlungen mit erforderlicher Nachsorge

In [77]:
follow_up = treatment.merge(
    patients[["Patienten_ID", "Vorname", "Nachname"]], on="Patienten_ID"
).merge(services[["Leistungs_ID", "Leistungsname"]], on="Leistungs_ID")

follow_up = follow_up[follow_up["Nachsorge_erforderlich"] == "Ja"][
    ["Vorname", "Nachname", "Leistungsname", "Nachsorge_erforderlich"]
]

follow_up

,Vorname,Nachname,Leistungsname,Nachsorge_erforderlich
1,Artem,Fedorov,Reflux-Operation,Ja
3,Alexander,Yakovlev,Nierenwerte,Ja
5,Igor,Mikhailov,EEG-Untersuchung,Ja
8,Anastasia,Orlov,Handchirurgie,Ja
9,Vera,Petrov,Allergie-Test,Ja
...,...,...,...,...
8726,Anna,Makarov,PET-CT,Ja
8741,Marina,Yakovlev,Nierenwerte,Ja
8745,Irina,Morozov,Arthroskopie Knie,Ja
8746,Svetlana,Popov,CT Thorax,Ja


10. Age groups and treatment preferences / Altersgruppen und Behandlungspräferenzen

In [78]:
patients["age_groups"] = pd.cut(
    patients["Alter_bei_Anmeldung"],
    bins=[-float("inf"), 29, 50, 70, float("inf")],
    labels=["Under 30", "30-50", "51-70", "Over 70"],
)

df = patients.merge(treatment, on="Patienten_ID").merge(
    services[["Leistungs_ID", "Kategorie"]], on="Leistungs_ID"
)

age_treatment = (
    df.groupby(["age_groups", "Kategorie"], as_index=False)
    .size()
    .rename(columns={"size": "treatment_counts"})
    .sort_values(
        ["Kategorie", "age_groups", "treatment_counts"], ascending=[True, True, True]
    )
)

age_treatment[["Kategorie", "age_groups", "treatment_counts"]]

/var/folders/5r/q_qx2bp160x59nxx8z069jwm0000gn/T/ipykernel_6297/3281313711.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["age_groups", "Kategorie"], as_index=False)


,Kategorie,age_groups,treatment_counts
0,Augenheilkunde,Under 30,258
8,Augenheilkunde,30-50,253
16,Augenheilkunde,51-70,216
24,Augenheilkunde,Over 70,224
1,Gastroenterologie,Under 30,254
9,Gastroenterologie,30-50,251
17,Gastroenterologie,51-70,243
25,Gastroenterologie,Over 70,203
2,Kardiologie,Under 30,299
10,Kardiologie,30-50,286


11. Monthly treatment volume / Monatliches Behandlungsvolumen

In [79]:
monthly_volume = (
    treatment.assign(
        year=lambda x: x["Behandlungsdatum"].dt.year,
        month=lambda x: x["Behandlungsdatum"].dt.month,
    )
    .groupby(["year", "month"], as_index=False)
    .size()
    .rename(columns={"size": "treatment_count"})
    .sort_values(["year", "month"])
)

monthly_volume

,year,month,treatment_count
0,2015,1,8
1,2015,2,23
2,2015,3,49
3,2015,4,45
4,2015,5,63
...,...,...,...
117,2024,10,59
118,2024,11,72
119,2024,12,60
120,2025,1,69


12. Patient satisfaction by service category / Patientenzufriedenheit nach Leistungskategorie

In [81]:
satisfaction = (
    treatment.merge(services[["Leistungs_ID", "Kategorie"]], on="Leistungs_ID")
    .groupby(["Kategorie", "Patientenzufriedenheit"], as_index=False)
    .size()
    .rename(columns={"size": "satisfaction_by_category"})
    .sort_values(["Kategorie", "satisfaction_by_category"], ascending=[True, False])
)

satisfaction

,Kategorie,Patientenzufriedenheit,satisfaction_by_category
0,Augenheilkunde,Neutral,266
2,Augenheilkunde,Unzufrieden,250
1,Augenheilkunde,Sehr_zufrieden,226
3,Augenheilkunde,Zufrieden,209
5,Gastroenterologie,Sehr_zufrieden,260
4,Gastroenterologie,Neutral,234
7,Gastroenterologie,Zufrieden,230
6,Gastroenterologie,Unzufrieden,227
11,Kardiologie,Zufrieden,283
10,Kardiologie,Unzufrieden,280


13. Treatment duration vs cost analysis / Behandlungsdauer vs. Kostenanalyse

In [82]:
duration_cost = (
    services.merge(treatment, on="Leistungs_ID")
    .groupby("Dauer_Stunden", as_index=False)
    .agg(
        avg_costs=("Gesamtkosten_EUR", "mean"),
        treatments_count=("Gesamtkosten_EUR", "count"),
    )
    .round({"avg_costs": 2})
    .sort_values("Dauer_Stunden", ascending=False)
)

duration_cost

,Dauer_Stunden,avg_costs,treatments_count
11,12,26844.66,328
10,11,26863.70,242
9,10,26179.11,214
8,9,26401.14,119
7,8,25830.85,901
6,7,25950.48,734
5,6,27209.20,769
4,5,26133.83,624
3,4,26804.81,569
2,3,26449.94,1190


14. Patients without treatments / Patienten ohne Behandlungen

In [85]:
patients_no_treatment = patients.merge(
    treatment[["Patienten_ID"]], on="Patienten_ID", how="left", indicator=True
).loc[lambda x: x["_merge"] == "left_only", ["Vorname", "Nachname", "Anmeldedatum"]]

patients_no_treatment.sort_values("Anmeldedatum", ascending=True)

,Vorname,Nachname,Anmeldedatum
435,Ekaterina,Lebedev,2015-01-02
748,Oksana,Fedorov,2015-01-07
136,Alexander,Volkov,2015-01-08
326,Sergei,Stepanov,2015-01-09
338,Maxim,Sidorov,2015-01-09
...,...,...,...
15324,Anastasia,Kozlov,2024-12-28
14510,Kirill,Kozlov,2024-12-29
14980,Tatiana,Volkov,2024-12-29
15042,Pavel,Andreev,2024-12-30


15. Top 10 most expensive individual treatments / Top 10 teuerste Einzelbehandlungen

In [87]:
top_10_expensive = (
    treatment.merge(
        patients[["Patienten_ID", "Vorname", "Nachname"]], on="Patienten_ID"
    )
    .merge(services[["Leistungs_ID", "Leistungsname"]], on="Leistungs_ID")
    .sort_values("Gesamtkosten_EUR", ascending=False)
    .loc[
        :,
        [
            "Vorname",
            "Nachname",
            "Leistungsname",
            "Gesamtkosten_EUR",
            "Behandlungsdatum",
        ],
    ]
).head(10)

top_10_expensive

,Vorname,Nachname,Leistungsname,Gesamtkosten_EUR,Behandlungsdatum
2888,Nina,Kozlov,Schielen-Korrektur,51376,2018-10-18
4545,Vera,Lebedev,Migraine-Behandlung,51201,2020-12-19
1558,Konstantin,Smirnov,CT Thorax,50933,2017-12-21
3350,Yuri,Makarov,Neurochirurgische Operation,50903,2019-04-14
5138,Irina,Andreev,Neurochirurgische Operation,50802,2021-05-18
6411,Ekaterina,Smirnov,Allergie-Test,50755,2022-05-29
4904,Vladimir,Yakovlev,Angiographie,50578,2021-01-07
4763,Galina,Stepanov,Tumorentfernung,50386,2020-08-30
7931,Alexander,Orlov,Laser-Behandlung,50373,2023-10-11
723,Andrei,Mikhailov,Schilddruesenwerte,50366,2016-02-26


16. Insurance type distribution by country / Versicherungstyp-Verteilung nach Land

In [88]:
insurance_by_country = (
    patients.groupby(["Herkunftsland", "Versicherungstyp"], as_index=False)
    .size()
    .rename(columns={"size": "patients_by_country"})
    .sort_values(["Herkunftsland", "patients_by_country"], ascending=[True, False])
)

insurance_by_country

,Herkunftsland,Versicherungstyp,patients_by_country
1,Armenien,Gesetzlich,31
0,Armenien,Ausland,27
3,Armenien,Selbstzahler,26
2,Armenien,Privat,22
6,Belarus,Privat,240
5,Belarus,Gesetzlich,236
7,Belarus,Selbstzahler,233
4,Belarus,Ausland,216
10,Estland,Privat,103
8,Estland,Ausland,95


17. Treatment complications by service type / Behandlungskomplikationen nach Leistungstyp

In [89]:
complications = (
    treatment.merge(services[["Leistungs_ID", "Kategorie"]], on="Leistungs_ID")
    .groupby("Kategorie", as_index=False)
    .agg(
        with_complications=("Komplikationen", lambda x: (x == "Ja").sum()),
        total_treatments=("Komplikationen", "count"),
    )
)

complications["complications_rate_pct"] = (
    complications["with_complications"] * 100 / complications["total_treatments"]
).round(2)

complications

,Kategorie,with_complications,total_treatments,complications_rate_pct
0,Augenheilkunde,474,951,49.84
1,Gastroenterologie,478,951,50.26
2,Kardiologie,532,1113,47.80
3,Labordiagnostik,553,1115,49.60
4,Neurologie,587,1154,50.87
5,Onkologie,506,1044,48.47
6,Orthopaedie,627,1227,51.10
7,Radiologie,588,1195,49.21


18. Seasonal treatment patterns / Saisonale Behandlungsmuster

In [91]:
seasonal = (
    treatment.merge(services[["Leistungs_ID", "Kategorie"]], on="Leistungs_ID")
    .assign(quarter=lambda x: x["Behandlungsdatum"].dt.quarter)
    .groupby(["quarter", "Kategorie"], as_index=False)
    .size()
    .rename(columns={"size": "total_treatment"})
    .sort_values(["quarter", "total_treatment"], ascending=[True, False])
)

seasonal

,quarter,Kategorie,total_treatment
4,1,Neurologie,299
6,1,Orthopaedie,292
2,1,Kardiologie,280
7,1,Radiologie,270
3,1,Labordiagnostik,252
1,1,Gastroenterologie,250
5,1,Onkologie,248
0,1,Augenheilkunde,221
15,2,Radiologie,319
14,2,Orthopaedie,296


19. Average interpreter hours by language / Durchschnittliche Dolmetscherstunden nach Sprache

In [92]:
interpreter_stats = (
    patients.merge(treatment[["Patienten_ID", "Dolmetscherstunden"]], on="Patienten_ID")
    .groupby("Sprache", as_index=False)
    .agg(
        avg_interpreter_hours=("Dolmetscherstunden", "mean"),
        total_treatments=("Dolmetscherstunden", "count"),
    )
    .round({"avg_interpreter_hours": 2})
)

interpreter_stats

,Sprache,avg_interpreter_hours,total_treatments
0,Russisch,11.05,8750


20. Treatment cost breakdown / Aufschlüsselung der Behandlungskosten

In [97]:
costs = pd.Series(
    {
        "avg_service_costs": treatment["Leistungskosten_EUR"].mean(),
        "avg_interpreter_costs": treatment["Dolmetscherkosten_EUR"].mean(),
        "avg_transport_costs": treatment["Transportkosten_EUR"].mean(),
        "avg_accommodations_costs": treatment["Unterkunftskosten_EUR"].mean(),
        "avg_additional_costs": treatment["Zusatzkosten_EUR"].mean(),
    }
).round(2)

print(costs)

avg_service_costs           22543.64
avg_interpreter_costs         851.54
avg_transport_costs           421.91
avg_accommodations_costs     1654.68
avg_additional_costs         1000.77
dtype: float64


## Intermediate Level | Part 2

21. Patient demographics by education level

In [98]:
education_stats = (
    patients.groupby("Bildungsgrad", as_index=False)
    .agg(
        avg_age=("Alter_bei_Anmeldung", "mean"), patient_count=("Patienten_ID", "count")
    )
    .round({"avg_age": 2})
)

education_stats.head()

,Bildungsgrad,avg_age,patient_count
0,Abitur,47.44,3063
1,Grundschule,47.95,3071
2,Hochschulabschluss,47.41,3032
3,Mittlere Reife,47.19,3034


22. Treatment success rate by staff member

In [120]:
staff_success = (
    accompanying_staff.merge(
        treatment[["Mitarbeiter_ID", "Status"]], on="Mitarbeiter_ID"
    )
    .groupby(["Mitarbeiter_ID", "Vorname", "Nachname"], as_index=False)
    .agg(
        completed_treatment=("Status_y", lambda x: (x == "Abgeschlossen").sum()),
        total_treatments=("Status_y", "count"),
    )
)

staff_success

,Mitarbeiter_ID,Vorname,Nachname,completed_treatment,total_treatments
0,M001,Anna,Mueller,67,895
1,M002,Dmitri,Schmidt,80,866
2,M003,Elena,Weber,91,947
3,M004,Alexander,Wagner,62,875
4,M005,Svetlana,Fischer,76,861
5,M006,Pavel,Becker,74,852
6,M007,Marina,Schulz,64,866
7,M008,Viktor,Hoffmann,73,846
8,M009,Irina,Schaefer,82,882
9,M010,Sergei,Koch,67,860


23. Transportation preferences by country

In [121]:
transport_prefs = (
    patients.merge(treatment[["Patienten_ID", "Transportart"]], on="Patienten_ID")
    .groupby(["Herkunftsland", "Transportart"], as_index=False)
    .size()
    .rename(columns={"size": "treatment_count"})
    .sort_values(["Herkunftsland", "treatment_count"], ascending=[True, False])
)

transport_prefs

,Herkunftsland,Transportart,treatment_count
1,Armenien,Oeffentliche_Verkehrsmittel,20
3,Armenien,Taxi,19
2,Armenien,PKW,13
0,Armenien,Mietwagen,11
6,Belarus,PKW,186
7,Belarus,Taxi,165
5,Belarus,Oeffentliche_Verkehrsmittel,160
4,Belarus,Mietwagen,152
10,Estland,PKW,69
9,Estland,Oeffentliche_Verkehrsmittel,66


24. Medical reports by treatment status

In [122]:
reports_by_status = (
    treatment.merge(reports[["Behandlungs_ID", "Berichtstyp"]], on="Behandlungs_ID")
    .groupby(["Status", "Berichtstyp"], as_index=False)
    .size()
    .rename(columns={"size": "reports_count"})
    .sort_values(["Status", "reports_count"], ascending=[True, False])
)

reports_by_status

,Status,Berichtstyp,reports_count
2,Abgeschlossen,Laborergebnisse,58
0,Abgeschlossen,Behandlungsplan,57
5,Abgeschlossen,Operationsbericht,53
1,Abgeschlossen,Entlassungsbericht,52
7,Abgeschlossen,Voruntersuchung,51
...,...,...,...
89,Termin_vereinbart,Entlassungsbericht,55
90,Termin_vereinbart,Laborergebnisse,54
91,Termin_vereinbart,Nachsorgeplan,51
95,Termin_vereinbart,Voruntersuchung,51


25. Services requiring anesthesia analysis

In [123]:
narcose_services = services.groupby("Kategorie", as_index=False).agg(
    with_narcose=("Narkose_erforderlich", lambda x: (x == "Ja").sum()),
    total_services=("Narkose_erforderlich", "count"),
)

narcose_services

,Kategorie,with_narcose,total_services
0,Augenheilkunde,3,9
1,Gastroenterologie,4,9
2,Kardiologie,7,10
3,Labordiagnostik,6,10
4,Neurologie,6,10
5,Onkologie,6,10
6,Orthopaedie,7,11
7,Radiologie,5,11


26. Patient family history patterns

In [124]:
family_history = (
    patients.merge(treatment[["Patienten_ID", "Leistungs_ID"]], on="Patienten_ID")
    .merge(services[["Leistungs_ID", "Kategorie"]], on="Leistungs_ID")
    .groupby(["Familienanamnese", "Kategorie"], as_index=False)
    .size()
    .rename(columns={"size": "treatment_count"})
    .sort_values(["Familienanamnese", "Kategorie"])
)

family_history

,Familienanamnese,Kategorie,treatment_count
0,Autoimmunerkrankungen,Augenheilkunde,107
1,Autoimmunerkrankungen,Gastroenterologie,99
2,Autoimmunerkrankungen,Kardiologie,132
3,Autoimmunerkrankungen,Labordiagnostik,146
4,Autoimmunerkrankungen,Neurologie,121
...,...,...,...
67,Psychische Erkrankungen,Labordiagnostik,130
68,Psychische Erkrankungen,Neurologie,131
69,Psychische Erkrankungen,Onkologie,120
70,Psychische Erkrankungen,Orthopaedie,148


27. Treatment timeline analysis

In [125]:
timeline = patients.merge(
    treatment[
        [
            "Patienten_ID",
            "Anfragedatum",
            "Beratungsdatum",
            "Behandlungsdatum",
            "Abschlussdatum",
        ]
    ],
    on="Patienten_ID",
).query("Behandlungsdatum.notnull()")[
    [
        "Vorname",
        "Nachname",
        "Anfragedatum",
        "Beratungsdatum",
        "Behandlungsdatum",
        "Abschlussdatum",
    ]
]

timeline

,Vorname,Nachname,Anfragedatum,Beratungsdatum,Behandlungsdatum,Abschlussdatum
0,Anna,Lebedev,2015-07-12,2015-07-18,2015-08-22,2015-08-28
1,Marina,Popov,2015-06-09,2015-06-11,2015-07-03,2015-07-16
2,Vera,Kozlov,2015-12-28,2016-01-10,2016-02-19,2016-03-11
3,Vera,Kozlov,2015-05-03,2015-05-17,2015-07-02,2015-07-07
4,Igor,Kozlov,2015-03-20,2015-03-31,2015-05-21,2015-05-22
...,...,...,...,...,...,...
8745,Oksana,Stepanov,2024-09-20,2024-09-23,2024-10-09,2024-10-11
8746,Alexei,Ivanov,2024-08-20,2024-08-23,2024-09-23,2024-09-28
8747,Alexei,Ivanov,2024-02-19,2024-02-27,2024-03-22,2024-03-24
8748,Alexei,Ivanov,2024-01-26,2024-02-07,2024-02-21,2024-03-20


28. Cost efficiency by service complexity

In [126]:
complexity_costs = (
    services.merge(treatment[["Leistungs_ID", "Gesamtkosten_EUR"]], on="Leistungs_ID")
    .groupby("Komplexitaet", as_index=False)
    .agg(
        avg_costs=("Gesamtkosten_EUR", "mean"),
        treatment_count=("Gesamtkosten_EUR", "count"),
    )
    .round({"avg_costs": 2})
)

complexity_costs

,Komplexitaet,avg_costs,treatment_count
0,Hoch,26586.36,3120
1,Mittel,26783.73,2604
2,Niedrig,26087.37,3026


29. Patient retention analysis

In [127]:
retention = (
    patients.merge(
        treatment[["Patienten_ID", "Behandlungs_ID"]],
        on="Patienten_ID",
        how="left",
    )
    .groupby("Herkunftsland", as_index=False)
    .agg(
        unique_patients=("Patienten_ID", "nunique"),
        total_treatments=("Behandlungs_ID", "count"),
    )
)

retention

,Herkunftsland,unique_patients,total_treatments
0,Armenien,106,63
1,Belarus,925,663
2,Estland,386,250
3,Georgien,225,161
4,Kasachstan,839,648
5,Lettland,495,353
6,Litauen,351,270
7,Moldawien,247,202
8,Russland,5566,4006
9,Ukraine,3060,2134


30. Quality level vs patient satisfaction

In [128]:
quality_vs_satisfaction = (
    services.merge(
        treatment[["Leistungs_ID", "Patientenzufriedenheit", "Gesamtkosten_EUR"]],
        on="Leistungs_ID",
    )
    .groupby(["Qualitaetsstufe", "Patientenzufriedenheit"], as_index=False)
    .agg(
        total_treatments=("Gesamtkosten_EUR", "count"),
        avg_costs=("Gesamtkosten_EUR", "mean"),
    )
    .round({"avg_costs": 2})
    .sort_values(["Qualitaetsstufe", "avg_costs"], ascending=[True, False])
)

quality_vs_satisfaction

,Qualitaetsstufe,Patientenzufriedenheit,total_treatments,avg_costs
0,Luxus,Neutral,657,26853.73
3,Luxus,Zufrieden,620,26648.70
1,Luxus,Sehr_zufrieden,667,26645.52
2,Luxus,Unzufrieden,680,25999.20
6,Premium,Unzufrieden,917,27006.02
7,Premium,Zufrieden,929,26829.99
5,Premium,Sehr_zufrieden,946,26568.09
4,Premium,Neutral,980,25946.00
11,Standard,Zufrieden,584,26512.38
8,Standard,Neutral,625,26327.32


31. Staff language skills utilization

In [129]:
staff_language_util = (
    accompanying_staff.merge(
        treatment[["Mitarbeiter_ID", "Behandlungs_ID", "Dolmetscherstunden"]],
        on="Mitarbeiter_ID",
    )
    .groupby("Sprachkombination", as_index=False)
    .agg(
        total_treatments=("Behandlungs_ID", "count"),
        avg_translate_time=("Dolmetscherstunden", "mean"),
    )
    .round({"avg_translate_time": 2})
)

staff_language_util

,Sprachkombination,total_treatments,avg_translate_time
0,Russisch-Deutsch,8750,11.05


32. Treatment type by age and gender

In [131]:
patients["age_groups"] = pd.cut(
    patients["Alter_bei_Anmeldung"],
    bins=[-float("inf"), 39, 60, float("inf")],
    labels=["Jung", "Mittleres Alter", "Aelter"],
)

df = patients.merge(
    treatment[["Patienten_ID", "Leistungs_ID"]], on="Patienten_ID"
).merge(services[["Leistungs_ID", "Kategorie"]], on="Leistungs_ID")

treatment_by_age_gender = (
    df.groupby(["Geschlecht", "age_groups", "Kategorie"], as_index=False)
    .size()
    .rename(columns={"size": "total_treatments"})
    .sort_values(
        ["Geschlecht", "age_groups", "total_treatments"],
        ascending=[False, False, False],
    )
)

treatment_by_age_gender

/var/folders/5r/q_qx2bp160x59nxx8z069jwm0000gn/T/ipykernel_6297/1863651328.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["Geschlecht", "age_groups", "Kategorie"], as_index=False)


,Geschlecht,age_groups,Kategorie,total_treatments
46,W,Aelter,Orthopaedie,215
47,W,Aelter,Radiologie,214
42,W,Aelter,Kardiologie,203
40,W,Aelter,Augenheilkunde,201
43,W,Aelter,Labordiagnostik,194
44,W,Aelter,Neurologie,189
45,W,Aelter,Onkologie,184
41,W,Aelter,Gastroenterologie,179
36,W,Mittleres Alter,Neurologie,170
38,W,Mittleres Alter,Orthopaedie,162


33. Follow-up care requirements by service

In [132]:
followup_by_service = (
    services.merge(
        treatment[["Leistungs_ID", "Nachsorge_erforderlich"]],
        on="Leistungs_ID",
    )
    .groupby(
        ["Leistungsname", "Nachsorge_erforderlich_x", "Nachsorge_erforderlich_y"],
        as_index=False,
    )
    .size()
    .rename(
        columns={
            "Leistungsname": "service_name",
            "Nachsorge_erforderlich_x": "service_followup",
            "Nachsorge_erforderlich_y": "actual_followup",
            "size": "total_treatment",
        }
    )
)

followup_by_service

,service_name,service_followup,actual_followup,total_treatment
0,Allergie-Test,Ja,Ja,39
1,Allergie-Test,Ja,Nein,39
2,Allergie-Test,Ja,Teilweise,29
3,Angiographie,Ja,Ja,44
4,Angiographie,Ja,Nein,38
...,...,...,...,...
235,Vitaminanalyse,Nein,Nein,42
236,Vitaminanalyse,Nein,Teilweise,25
237,Wirbelsaeulen-Operation,Ja,Ja,33
238,Wirbelsaeulen-Operation,Ja,Nein,40


34. Revenue trends by quarter

In [133]:
quarterly_revenue = (
    treatment.loc[treatment["Behandlungsdatum"].notnull()]
    .assign(
        year=lambda x: x["Behandlungsdatum"].dt.year,
        quarter=lambda x: x["Behandlungsdatum"].dt.quarter,
    )
    .groupby(["year", "quarter"], as_index=False)
    .agg(
        total_costs=("Gesamtkosten_EUR", "sum"),
        total_treatments=("Behandlungsdatum", "count"),
    )
    .sort_values(["year", "quarter"])
)

quarterly_revenue

,year,quarter,total_costs,total_treatments
0,2015,1,1995736,80
1,2015,2,4382106,169
2,2015,3,4219032,157
3,2015,4,4282081,156
4,2016,1,5342380,185
5,2016,2,5065678,193
6,2016,3,4845818,190
7,2016,4,5100478,191
8,2017,1,5165734,194
9,2017,2,6095196,228


35. Treatment complexity vs duration analysis

In [134]:
complexity_analysis = (
    services.merge(
        treatment[["Leistungs_ID", "Begleitungstage", "Gesamtkosten_EUR"]],
        on="Leistungs_ID",
    )
    .groupby("Komplexitaet", as_index=False)
    .agg(
        avg_duration=("Dauer_Stunden", "mean"),
        avg_accompanying=("Begleitungstage", "mean"),
        avg_costs=("Gesamtkosten_EUR", "mean"),
    )
    .round({"avg_duration": 2, "avg_accompanying": 2, "avg_costs": 2})
)

complexity_analysis

,Komplexitaet,avg_duration,avg_accompanying,avg_costs
0,Hoch,4.87,8.04,26586.36
1,Mittel,3.86,7.96,26783.73
2,Niedrig,4.95,8.00,26087.37


36. Patient education level vs treatment choices

In [135]:
education_vs_quality = (
    patients.merge(
        treatment[["Patienten_ID", "Leistungs_ID", "Gesamtkosten_EUR"]],
        on="Patienten_ID",
    )
    .merge(services[["Leistungs_ID", "Qualitaetsstufe"]], on="Leistungs_ID")
    .groupby(["Bildungsgrad", "Qualitaetsstufe"], as_index=False)
    .agg(
        total_treatment=("Gesamtkosten_EUR", "count"),
        avg_costs=("Gesamtkosten_EUR", "mean"),
    )
    .round({"avg_costs": 2})
)

education_vs_quality

,Bildungsgrad,Qualitaetsstufe,total_treatment,avg_costs
0,Abitur,Luxus,684,26701.00
1,Abitur,Premium,1017,26121.61
2,Abitur,Standard,554,25699.79
3,Grundschule,Luxus,636,26505.14
4,Grundschule,Premium,914,26265.50
5,Grundschule,Standard,652,26702.24
6,Hochschulabschluss,Luxus,637,26922.17
7,Hochschulabschluss,Premium,974,26819.49
8,Hochschulabschluss,Standard,601,25488.30
9,Mittlere Reife,Luxus,667,26007.40


37. Staff experience vs treatment outcomes

In [136]:
staff_experience = (
    accompanying_staff.merge(
        treatment[["Mitarbeiter_ID", "Patientenzufriedenheit"]],
        on="Mitarbeiter_ID",
    )
    .groupby("Berufserfahrung_Jahre", as_index=False)
    .agg(
        satisfaction_rate=(
            "Patientenzufriedenheit",
            lambda x: round((x == "Zufrieden").mean(), 2),
        ),
        total_treatments=("Patientenzufriedenheit", "count"),
    )
    .sort_values("Berufserfahrung_Jahre")
)

staff_experience

,Berufserfahrung_Jahre,satisfaction_rate,total_treatments
0,3,0.23,882
1,4,0.25,1756
2,5,0.26,866
3,7,0.28,875
4,10,0.24,866
5,13,0.23,846
6,15,0.24,2659


38. Treatment location preferences by insurance type

In [137]:
location_by_insurance = (
    patients.merge(
        treatment[["Patienten_ID", "Behandlungsort", "Gesamtkosten_EUR"]],
        on="Patienten_ID",
    )
    .groupby(["Versicherungstyp", "Behandlungsort"], as_index=False)
    .agg(
        total_treatments=("Gesamtkosten_EUR", "count"),
        avg_costs=("Gesamtkosten_EUR", "mean"),
    )
    .round({"avg_costs": 2})
    .sort_values(["Versicherungstyp", "avg_costs"], ascending=[True, False])
)

location_by_insurance

,Versicherungstyp,Behandlungsort,total_treatments,avg_costs
3,Ausland,Dortmund,136,28785.35
7,Ausland,Essen,149,27752.54
6,Ausland,Duisburg,128,27286.71
13,Ausland,Nuremberg,133,27013.40
10,Ausland,Hannover,130,26781.62
8,Ausland,Frankfurt,142,26740.75
12,Ausland,Munich,145,26331.30
14,Ausland,Stuttgart,150,26073.54
1,Ausland,Bremen,129,26055.12
0,Ausland,Berlin,137,25848.91


39. Marital status impact on treatment decisions

In [138]:
marital_impact = (
    patients.merge(
        treatment[["Patienten_ID", "Leistungs_ID", "Gesamtkosten_EUR"]],
        on="Patienten_ID",
    )
    .merge(services[["Leistungs_ID", "Kategorie"]], on="Leistungs_ID")
    .groupby(["Familienstand", "Kategorie"], as_index=False)
    .agg(
        anzahl_behandlungen=("Gesamtkosten_EUR", "count"),
        avg_costs=("Gesamtkosten_EUR", "mean"),
    )
    .round({"avg_costs": 2})
)

marital_impact

,Familienstand,Kategorie,anzahl_behandlungen,avg_costs
0,Geschieden,Augenheilkunde,239,24809.49
1,Geschieden,Gastroenterologie,205,26657.00
2,Geschieden,Kardiologie,303,27691.89
3,Geschieden,Labordiagnostik,292,26085.50
4,Geschieden,Neurologie,288,26701.26
5,Geschieden,Onkologie,264,24982.52
6,Geschieden,Orthopaedie,293,25237.67
7,Geschieden,Radiologie,292,26330.98
8,Ledig,Augenheilkunde,237,26680.19
9,Ledig,Gastroenterologie,230,28614.68


40. Previous treatment experience impact

In [139]:
prev_treatment_impact = (
    patients.merge(
        treatment[["Patienten_ID", "Gesamtkosten_EUR", "Patientenzufriedenheit"]],
        on="Patienten_ID",
    )
    .groupby("Vorherige_Behandlung_Deutschland", as_index=False)
    .agg(
        avg_costs=("Gesamtkosten_EUR", "mean"),
        satisfaction_rate=(
            "Patientenzufriedenheit",
            lambda x: round((x == "Zufrieden").mean(), 2),
        ),
        total_treatments=("Gesamtkosten_EUR", "count"),
    )
    .round({"avg_costs": 2})
)

prev_treatment_impact

,Vorherige_Behandlung_Deutschland,avg_costs,satisfaction_rate,total_treatments
0,Ja,26597.99,0.24,4398
1,Nein,26345.75,0.25,4352


## Advanced Level

1. Complex patient journey analysis with multiple CTEs / Komplexe Analyse des Patientenpfads mit mehreren CTEs

In [141]:
patient_stats = (
    patients.merge(
        treatment[
            [
                "Patienten_ID",
                "Behandlungs_ID",
                "Gesamtkosten_EUR",
                "Anfragedatum",
                "Abschlussdatum",
            ]
        ],
        on="Patienten_ID",
        how="left",
    )
    .groupby(["Patienten_ID", "Herkunftsland", "Alter_bei_Anmeldung"], as_index=False)
    .agg(
        total_treatments=("Behandlungs_ID", "count"),
        avg_cost=("Gesamtkosten_EUR", "mean"),
        first_inquiry=("Anfragedatum", "min"),
        last_completion=("Abschlussdatum", "max"),
    )
)

country_benchmarks = patient_stats.groupby("Herkunftsland", as_index=False).agg(
    country_avg_treatments=("total_treatments", "mean"),
    country_avg_cost=("avg_cost", "mean"),
)

result = patient_stats.merge(country_benchmarks, on="Herkunftsland")
result = result[result["total_treatments"] > 0]
result["treatment_frequency_category"] = result.apply(
    lambda x: "Above Average"
    if x["total_treatments"] > x["country_avg_treatments"]
    else (
        "Average"
        if x["total_treatments"] == x["country_avg_treatments"]
        else "Below Average"
    ),
    axis=1,
)

result

,Patienten_ID,Herkunftsland,Alter_bei_Anmeldung,total_treatments,avg_cost,first_inquiry,last_completion,country_avg_treatments,country_avg_cost,treatment_frequency_category
0,P00001,Russland,13,1,30335.000000,2015-07-12,2015-08-28,0.719727,26167.366861,Above Average
2,P00003,Belarus,53,1,37280.000000,2015-06-09,2015-07-16,0.716757,26824.090369,Above Average
3,P00004,Ukraine,15,2,36145.000000,2015-05-03,2016-03-11,0.697386,26568.942135,Above Average
4,P00005,Russland,47,1,47544.000000,2015-03-20,2015-05-22,0.719727,26167.366861,Above Average
5,P00006,Russland,44,1,17940.000000,2015-07-14,2015-08-05,0.719727,26167.366861,Above Average
...,...,...,...,...,...,...,...,...,...,...
8743,P08744,Russland,19,1,29306.000000,2024-10-13,2024-12-10,0.719727,26167.366861,Above Average
8745,P08746,Ukraine,23,1,19934.000000,2024-01-17,2024-02-15,0.697386,26568.942135,Above Average
8746,P08747,Russland,87,2,20264.000000,2024-09-20,2024-12-30,0.719727,26167.366861,Above Average
8748,P08749,Russland,62,3,42312.333333,2024-01-26,2024-09-28,0.719727,26167.366861,Above Average


2. Window functions for ranking and running totals / Fensterfunktionen für Rangfolgen und kumulierte Summen


In [142]:
df = patients.merge(
    treatment[
        ["Patienten_ID", "Behandlungsdatum", "Gesamtkosten_EUR", "Behandlungs_ID"]
    ],
    on="Patienten_ID",
).dropna(subset=["Behandlungsdatum"])

df = df.sort_values(["Herkunftsland", "Gesamtkosten_EUR"], ascending=[True, False])

df["cost_rank_in_country"] = df.groupby("Herkunftsland")["Gesamtkosten_EUR"].rank(
    method="first", ascending=False
)

df["overall_cost_rank"] = df["Gesamtkosten_EUR"].rank(method="min", ascending=False)

df = df.sort_values(["Herkunftsland", "Behandlungsdatum"])
df["running_total_by_country"] = df.groupby("Herkunftsland")[
    "Gesamtkosten_EUR"
].cumsum()

df["previous_treatment_cost"] = (
    df.sort_values("Behandlungsdatum")
    .groupby("Patienten_ID")["Gesamtkosten_EUR"]
    .shift(1)
)

df["next_treatment_date"] = (
    df.sort_values("Behandlungsdatum")
    .groupby("Patienten_ID")["Behandlungsdatum"]
    .shift(-1)
)

result = df[
    [
        "Herkunftsland",
        "Nachname",
        "Behandlungsdatum",
        "Gesamtkosten_EUR",
        "cost_rank_in_country",
        "overall_cost_rank",
        "running_total_by_country",
        "previous_treatment_cost",
        "next_treatment_date",
    ]
]

result

,Herkunftsland,Nachname,Behandlungsdatum,Gesamtkosten_EUR,cost_rank_in_country,overall_cost_rank,running_total_by_country,previous_treatment_cost,next_treatment_date
535,Armenien,Morozov,2015-03-17,19516,40.0,5671.0,19516,NaN,NaT
485,Armenien,Ivanov,2015-07-17,33818,22.0,2988.0,53334,NaN,NaT
536,Armenien,Sidorov,2016-01-03,23151,34.0,4969.0,76485,NaN,NaT
858,Armenien,Popov,2016-10-15,38707,14.0,2020.0,115192,NaN,NaT
1099,Armenien,Petrov,2016-10-24,46648,7.0,455.0,161840,NaN,NaT
...,...,...,...,...,...,...,...,...,...
8187,Ukraine,Smirnov,2025-01-21,39423,467.0,1884.0,56430086,36230.0,2025-01-24
8186,Ukraine,Smirnov,2025-01-24,20728,1334.0,5446.0,56450814,39423.0,NaT
8709,Ukraine,Kozlov,2025-01-28,28249,984.0,4045.0,56479063,5645.0,NaT
8702,Ukraine,Sokolov,2025-01-31,24771,1125.0,4655.0,56503834,15724.0,NaT


 3. Advanced pivot analysis using conditional aggregation / Fortgeschrittene Pivot-Analyse mit bedingter Aggregation


In [143]:
demo = (
    patients.merge(
        treatment[["Patienten_ID", "Gesamtkosten_EUR"]],
        on="Patienten_ID",
        how="left",
    )
    .groupby("Herkunftsland", as_index=False)
    .agg(
        total_patients=("Patienten_ID", "count"),
        male_patients=("Geschlecht", lambda x: (x == "M").sum()),
        female_patients=("Geschlecht", lambda x: (x == "W").sum()),
        young_patients=("Alter_bei_Anmeldung", lambda x: (x < 30).sum()),
        middle_age_patients=(
            "Alter_bei_Anmeldung",
            lambda x: ((x >= 30) & (x <= 60)).sum(),
        ),
        senior_patients=("Alter_bei_Anmeldung", lambda x: (x > 60).sum()),
        private_insurance=("Versicherungstyp", lambda x: (x == "Privat").sum()),
        public_insurance=("Versicherungstyp", lambda x: (x == "Gesetzlich").sum()),
        self_pay=("Versicherungstyp", lambda x: (x == "Selbstzahler").sum()),
        avg_treatment_cost=("Gesamtkosten_EUR", lambda x: round(x.dropna().mean(), 2)),
    )
)

demo = demo[demo["total_patients"] >= 100].sort_values(
    "total_patients", ascending=False
)

demo

,Herkunftsland,total_patients,male_patients,female_patients,young_patients,middle_age_patients,senior_patients,private_insurance,public_insurance,self_pay,avg_treatment_cost
8,Russland,6995,3455,3540,1931,2615,2449,1760,1771,1778,26325.73
9,Ukraine,3842,1938,1904,1046,1463,1333,980,977,970,26487.98
1,Belarus,1190,567,623,354,463,373,305,306,309,26935.34
4,Kasachstan,1095,524,571,312,414,369,276,255,287,26242.61
5,Lettland,628,315,313,185,226,217,164,138,156,26165.49
2,Estland,476,224,252,133,188,155,127,111,117,27524.42
6,Litauen,451,233,218,117,162,172,121,137,93,26813.20
7,Moldawien,328,153,175,84,130,114,86,81,66,26735.91
3,Georgien,289,136,153,76,115,98,68,78,67,26779.44
0,Armenien,123,64,59,36,51,36,29,35,28,27235.41


4. Complex subquery with EXISTS and correlated subqueries / Komplexe Unterabfrage mit EXISTS und korrelierten Unterabfragen

In [144]:
country_avg_cost = (
    patients.merge(treatment[["Patienten_ID", "Gesamtkosten_EUR"]], on="Patienten_ID")
    .groupby("Herkunftsland")["Gesamtkosten_EUR"]
    .mean()
    * 1.5
)

not_abgebrochen = treatment.groupby("Patienten_ID")["Status"].apply(
    lambda x: (x == "Abgebrochen").sum() == 0
)


def patient_filter(row):
    pid = row["Patienten_ID"]
    country = row["Herkunftsland"]
    high_cost = treatment[
        (treatment["Patienten_ID"] == pid)
        & (treatment["Gesamtkosten_EUR"] > country_avg_cost.get(country, float("inf")))
    ]
    no_abgebrochen = not_abgebrochen.get(pid, True)
    return len(high_cost) > 0 and no_abgebrochen


treatments = (
    treatment.groupby("Patienten_ID")
    .agg(
        total_treatments=("Behandlungs_ID", "count"),
        avg_cost=("Gesamtkosten_EUR", "mean"),
    )
    .reset_index()
)

result = patients.merge(treatments, on="Patienten_ID")
result = result[result.apply(patient_filter, axis=1)]
result = result[
    [
        "Patienten_ID",
        "Vorname",
        "Nachname",
        "Herkunftsland",
        "total_treatments",
        "avg_cost",
    ]
]

result

,Patienten_ID,Vorname,Nachname,Herkunftsland,total_treatments,avg_cost
2,P00004,Vera,Kozlov,Ukraine,2,36145.000000
3,P00005,Igor,Kozlov,Russland,1,47544.000000
5,P00007,Igor,Ivanov,Russland,1,48295.000000
8,P00011,Larisa,Smirnov,Russland,2,33750.500000
14,P00019,Vladimir,Stepanov,Russland,3,26905.333333
...,...,...,...,...,...,...
5520,P08729,Elena,Makarov,Russland,2,30395.500000
5521,P08730,Oksana,Sokolov,Russland,3,33570.000000
5526,P08742,Mikhail,Andreev,Russland,1,41646.000000
5531,P08749,Alexei,Ivanov,Russland,3,42312.333333


5. Advanced date calculations and temporal analysis / Fortgeschrittene Datumsberechnungen und zeitliche Analysen

In [146]:
grouped = (
    treatment.loc[
        treatment["Behandlungsdatum"].notnull()
        & (treatment["Behandlungsdatum"] >= "2015-01-01")
    ]
    .assign(
        treatment_year=lambda x: x["Behandlungsdatum"].dt.year.astype(str),
        treatment_month=lambda x: x["Behandlungsdatum"].dt.month.astype(str),
    )
    .groupby(["treatment_year", "treatment_month"])
)

advanced_dates = (
    grouped.apply(
        lambda df: pd.Series(
            {
                "treatments_count": df["Behandlungsdatum"].count(),
                "avg_days_inquiry_to_treatment": (
                    df["Behandlungsdatum"] - df["Anfragedatum"]
                ).dt.days.mean(),
                "avg_days_treatment_to_completion": (
                    df["Abschlussdatum"] - df["Behandlungsdatum"]
                ).dt.days.mean(),
                "avg_cost": df["Gesamtkosten_EUR"].mean(),
                "min_cost": df["Gesamtkosten_EUR"].min(),
                "max_cost": df["Gesamtkosten_EUR"].max(),
            }
        )
    )
    .reset_index()
    .sort_values(["treatment_year", "treatment_month"])
)

advanced_dates

/var/folders/5r/q_qx2bp160x59nxx8z069jwm0000gn/T/ipykernel_6297/2571176259.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(


,treatment_year,treatment_month,treatments_count,avg_days_inquiry_to_treatment,avg_days_treatment_to_completion,avg_cost,min_cost,max_cost
0,2015,1,8.0,18.375000,12.000000,34622.500000,15301.0,48400.0
1,2015,10,50.0,40.480000,16.420000,25519.360000,5851.0,47381.0
2,2015,11,52.0,40.384615,15.826923,28465.961538,4258.0,47262.0
3,2015,12,54.0,40.500000,16.888889,28257.092593,5771.0,47617.0
4,2015,2,23.0,30.869565,15.478261,25964.521739,4342.0,49200.0
...,...,...,...,...,...,...,...,...
117,2024,7,67.0,39.985075,15.582090,26161.731343,4400.0,47320.0
118,2024,8,52.0,39.057692,16.500000,27780.192308,4457.0,48656.0
119,2024,9,59.0,41.084746,15.677966,24179.457627,4604.0,48762.0
120,2025,1,69.0,45.492754,15.173913,26628.304348,4084.0,49498.0


6. Complex multi-table join with advanced filtering / Komplexes Join mehrerer Tabellen mit fortgeschrittener Filterung

In [147]:
filtered = (
    patients.merge(
        treatment[
            (treatment["Status"] == "Abgeschlossen")
            & (treatment["Behandlungsdatum"] >= "2015-01-01")
            & (treatment["Behandlungsdatum"] <= "2023-12-31")
        ],
        on="Patienten_ID",
    )
    .merge(
        services[services["Komplexitaet"].isin(["Hoch", "Sehr hoch"])][
            ["Leistungs_ID", "Kategorie", "Komplexitaet"]
        ],
        on="Leistungs_ID",
    )
    .merge(
        accompanying_staff[["Mitarbeiter_ID", "Sprachkombination"]], on="Mitarbeiter_ID"
    )
)

result = filtered.groupby(
    ["Herkunftsland", "Kategorie", "Sprachkombination"], as_index=False
).agg(
    unique_patients=("Patienten_ID", "nunique"),
    total_treatments=("Behandlungs_ID", "count"),
    avg_cost=("Gesamtkosten_EUR", "mean"),
    total_interpreter_costs=("Dolmetscherkosten_EUR", "sum"),
    avg_accompaniment_days=("Begleitungstage", "mean"),
)

result = result[result["total_treatments"] >= 5].sort_values(
    "total_treatments", ascending=False
)

result

,Herkunftsland,Kategorie,Sprachkombination,unique_patients,total_treatments,avg_cost,total_interpreter_costs,avg_accompaniment_days
40,Russland,Neurologie,Russisch-Deutsch,31,31,25400.677419,25654,7.774194
37,Russland,Gastroenterologie,Russisch-Deutsch,16,16,27205.687500,12162,6.687500
36,Russland,Augenheilkunde,Russisch-Deutsch,14,14,27729.357143,10702,6.357143
41,Russland,Onkologie,Russisch-Deutsch,12,12,31342.416667,9261,7.333333
43,Russland,Radiologie,Russisch-Deutsch,11,11,24861.363636,11235,5.363636
38,Russland,Kardiologie,Russisch-Deutsch,10,10,29293.200000,7062,8.700000
44,Ukraine,Augenheilkunde,Russisch-Deutsch,10,10,31959.800000,7252,8.300000
48,Ukraine,Neurologie,Russisch-Deutsch,9,10,21148.900000,7690,6.400000
51,Ukraine,Radiologie,Russisch-Deutsch,10,10,30227.000000,7627,8.300000
46,Ukraine,Kardiologie,Russisch-Deutsch,9,9,25584.777778,6827,8.333333


7. Advanced statistical analysis with variance calculations / Fortgeschrittener statistischer Analyse mit Varianzberechnung

In [148]:
cost_stats = (
    patients.merge(
        treatment[["Patienten_ID", "Leistungs_ID", "Gesamtkosten_EUR"]],
        on="Patienten_ID",
    )
    .merge(services[["Leistungs_ID", "Kategorie"]], on="Leistungs_ID")
    .dropna(subset=["Gesamtkosten_EUR"])
)

grouped = cost_stats.groupby(["Herkunftsland", "Kategorie"], as_index=False).agg(
    treatment_count=("Gesamtkosten_EUR", "count"),
    mean_cost=("Gesamtkosten_EUR", "mean"),
    min_cost=("Gesamtkosten_EUR", "min"),
    max_cost=("Gesamtkosten_EUR", "max"),
    mean_sq_cost=("Gesamtkosten_EUR", lambda x: (x**2).mean()),
)

grouped["variance_cost"] = grouped["mean_sq_cost"] - grouped["mean_cost"] ** 2
grouped = grouped[grouped["treatment_count"] >= 10]

grouped["std_dev_cost"] = grouped["variance_cost"].apply(lambda x: x**0.5)
grouped["cost_range"] = grouped["max_cost"] - grouped["min_cost"]
grouped["cost_variability_category"] = grouped.apply(
    lambda x: "Low Variability"
    if x["std_dev_cost"] / x["mean_cost"] < 0.2
    else (
        "Medium Variability"
        if 0.2 <= x["std_dev_cost"] / x["mean_cost"] <= 0.5
        else "High Variability"
    ),
    axis=1,
)

result = grouped.sort_values("variance_cost", ascending=False)[
    [
        "Herkunftsland",
        "Kategorie",
        "treatment_count",
        "mean_cost",
        "min_cost",
        "max_cost",
        "variance_cost",
        "std_dev_cost",
        "cost_variability_category",
        "cost_range",
    ]
]

result

,Herkunftsland,Kategorie,treatment_count,mean_cost,min_cost,max_cost,variance_cost,std_dev_cost,cost_variability_category,cost_range
24,Georgien,Augenheilkunde,17,25767.588235,5221,47822,2.284030e+08,15113.006226,High Variability,42601
31,Georgien,Radiologie,27,27394.407407,2291,48724,2.043545e+08,14295.260109,High Variability,46433
15,Belarus,Radiologie,105,28590.352381,3046,49988,1.994563e+08,14122.900197,Medium Variability,46942
10,Belarus,Kardiologie,90,26723.855556,5643,49011,1.984318e+08,14086.583862,High Variability,43368
19,Estland,Labordiagnostik,25,24236.280000,5538,49886,1.978451e+08,14065.741199,High Variability,44348
...,...,...,...,...,...,...,...,...,...,...
7,Armenien,Radiologie,10,25409.100000,10315,39530,1.287154e+08,11345.280961,Medium Variability,29215
44,Lettland,Neurologie,56,30323.928571,6020,48044,1.265787e+08,11250.721512,Medium Variability,42024
1,Armenien,Gastroenterologie,10,30282.000000,18338,49430,1.242927e+08,11148.663355,Medium Variability,31092
28,Georgien,Neurologie,14,25926.785714,6734,48463,1.148952e+08,10718.920096,Medium Variability,41729


8. Patient cohort analysis with retention metrics / Kohortenanalyse von Patienten mit Retentionsmetriken

In [152]:
patient_cohorts = (
    patients.merge(
        treatment[
            ["Patienten_ID", "Behandlungs_ID", "Behandlungsdatum", "Gesamtkosten_EUR"]
        ],
        on="Patienten_ID",
        how="left",
    )
    .assign(registration_cohort=lambda x: x["Anmeldedatum"].dt.strftime("%Y-%m"))
    .groupby(["Patienten_ID", "Herkunftsland", "registration_cohort"], as_index=False)
    .agg(
        first_treatment_date=("Behandlungsdatum", "min"),
        last_treatment_date=("Behandlungsdatum", "max"),
        total_treatments=("Behandlungs_ID", "count"),
        total_spent=("Gesamtkosten_EUR", "sum"),
    )
)

cohort_metrics = patient_cohorts.groupby(
    ["registration_cohort", "Herkunftsland"], as_index=False
).agg(
    cohort_size=("Patienten_ID", "count"),
    active_patients=("total_treatments", lambda x: (x > 0).sum()),
    repeat_patients=("total_treatments", lambda x: (x > 1).sum()),
    avg_treatments_per_patient=("total_treatments", "mean"),
    avg_revenue_per_patient=("total_spent", "mean"),
)

lifespan = (
    patient_cohorts.assign(
        lifespan_days=(
            patient_cohorts["last_treatment_date"]
            - patient_cohorts["first_treatment_date"]
        ).dt.days
    )
    .groupby(["registration_cohort", "Herkunftsland"])["lifespan_days"]
    .mean()
    .reset_index(name="avg_patient_lifespan_days")
)
cohort_metrics = cohort_metrics.merge(
    lifespan, on=["registration_cohort", "Herkunftsland"]
)


cohort_metrics = cohort_metrics[cohort_metrics["cohort_size"] >= 10].copy()
cohort_metrics["activation_rate_percent"] = (
    100 * cohort_metrics["active_patients"] / cohort_metrics["cohort_size"]
).round(2)
cohort_metrics["retention_rate_percent"] = (
    100
    * cohort_metrics["repeat_patients"]
    / cohort_metrics["active_patients"].replace(0, pd.NA)
).round(2)
cohort_metrics = cohort_metrics.sort_values(["registration_cohort", "Herkunftsland"])

cohort_metrics

,registration_cohort,Herkunftsland,cohort_size,active_patients,repeat_patients,avg_treatments_per_patient,avg_revenue_per_patient,avg_patient_lifespan_days,activation_rate_percent,retention_rate_percent
7,2015-01,Russland,24,19,8,1.166667,31231.500000,66.473684,79.17,42.105263
8,2015-01,Ukraine,16,8,3,0.687500,17374.687500,56.375000,50.00,37.5
17,2015-02,Russland,28,17,5,0.785714,20298.500000,51.352941,60.71,29.411765
18,2015-02,Ukraine,16,9,6,0.937500,26456.500000,50.555556,56.25,66.666667
26,2015-03,Russland,41,25,11,0.926829,26762.878049,57.440000,60.98,44.0
...,...,...,...,...,...,...,...,...,...,...
1070,2024-10,Ukraine,19,0,0,0.000000,0.000000,NaN,0.00,<NA>
1078,2024-11,Russland,34,0,0,0.000000,0.000000,NaN,0.00,<NA>
1079,2024-11,Ukraine,18,0,0,0.000000,0.000000,NaN,0.00,<NA>
1087,2024-12,Russland,39,0,0,0.000000,0.000000,NaN,0.00,<NA>


9. Treatment outcome prediction model data preparation / Datenvorbereitung für das Prognosemodell der Behandlungsergebnisse


In [153]:
model_data = (
    patients.merge(
        treatment[
            (treatment["Status"].isin(["Abgeschlossen", "Abgebrochen"]))
            & (treatment["Behandlungsdatum"].notnull())
        ][
            [
                "Patienten_ID",
                "Leistungs_ID",
                "Mitarbeiter_ID",
                "Begleitungstage",
                "Dolmetscherstunden",
                "Transportart",
                "Behandlungsort",
                "Status",
                "Patientenzufriedenheit",
                "Gesamtkosten_EUR",
                "Behandlungsdatum",
                "Anfragedatum",
                "Nachsorge_erforderlich",
            ]
        ],
        on="Patienten_ID",
    )
    .merge(
        services[
            [
                "Leistungs_ID",
                "Kategorie",
                "Komplexitaet",
                "Dauer_Stunden",
                "Narkose_erforderlich",
            ]
        ],
        on="Leistungs_ID",
    )
    .merge(
        accompanying_staff[
            ["Mitarbeiter_ID", "Berufserfahrung_Jahre", "Sprachkombination"]
        ],
        on="Mitarbeiter_ID",
    )
)

model_data = model_data.assign(
    treatment_category=model_data["Kategorie"],
    treatment_complexity=model_data["Komplexitaet"],
    treatment_duration=model_data["Dauer_Stunden"],
    anesthesia_required=model_data["Narkose_erforderlich"],
    staff_experience=model_data["Berufserfahrung_Jahre"],
    language_support=model_data["Sprachkombination"],
    accompaniment_days=model_data["Begleitungstage"],
    interpreter_hours=model_data["Dolmetscherstunden"],
    transport_type=model_data["Transportart"],
    treatment_location=model_data["Behandlungsort"],
    successful_outcome=(
        (model_data["Status"] == "Abgeschlossen")
        & (model_data["Patientenzufriedenheit"] == "Zufrieden")
    ).astype(int),
    total_cost=model_data["Gesamtkosten_EUR"],
    days_to_treatment=(
        (model_data["Behandlungsdatum"] - model_data["Anfragedatum"]).dt.days
    ),
    followup_required=(model_data["Nachsorge_erforderlich"] == "Ja").astype(int),
)

model_data = model_data[
    [
        "Patienten_ID",
        "Alter_bei_Anmeldung",
        "Geschlecht",
        "Herkunftsland",
        "Versicherungstyp",
        "Bildungsgrad",
        "Familienanamnese",
        "treatment_category",
        "treatment_complexity",
        "treatment_duration",
        "anesthesia_required",
        "staff_experience",
        "language_support",
        "accompaniment_days",
        "interpreter_hours",
        "transport_type",
        "treatment_location",
        "successful_outcome",
        "total_cost",
        "days_to_treatment",
        "followup_required",
    ]
]

model_data

,Patienten_ID,Alter_bei_Anmeldung,Geschlecht,Herkunftsland,Versicherungstyp,Bildungsgrad,Familienanamnese,treatment_category,treatment_complexity,treatment_duration,...,staff_experience,language_support,accompaniment_days,interpreter_hours,transport_type,treatment_location,successful_outcome,total_cost,days_to_treatment,followup_required
0,P00005,47,M,Russland,Selbstzahler,Abitur,Neurologische Erkrankungen,Kardiologie,Hoch,5,...,4,Russisch-Deutsch,15,14,PKW,Duesseldorf,1,47544,62,0
1,P00013,82,M,Kasachstan,Gesetzlich,Abitur,Erbliche Stoffwechselerkrankungen,Orthopaedie,Hoch,4,...,4,Russisch-Deutsch,4,3,Mietwagen,Nuremberg,1,35063,31,1
2,P00043,45,W,Russland,Ausland,Abitur,Bluthochdruck,Kardiologie,Hoch,4,...,15,Russisch-Deutsch,7,11,PKW,Stuttgart,0,26730,37,0
3,P00047,69,M,Russland,Privat,Mittlere Reife,Diabetes mellitus,Labordiagnostik,Niedrig,3,...,4,Russisch-Deutsch,14,13,Taxi,Nuremberg,0,37333,29,0
4,P00049,70,M,Ukraine,Privat,Mittlere Reife,Herz-Kreislauf-Erkrankungen,Kardiologie,Hoch,12,...,7,Russisch-Deutsch,15,5,PKW,Bremen,1,38183,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,P08714,87,M,Russland,Privat,Mittlere Reife,Herz-Kreislauf-Erkrankungen,Onkologie,Niedrig,7,...,3,Russisch-Deutsch,13,16,PKW,Cologne,0,11286,39,0
732,P08723,65,M,Russland,Gesetzlich,Abitur,Onkologische Erkrankungen,Gastroenterologie,Mittel,8,...,15,Russisch-Deutsch,2,4,Oeffentliche_Verkehrsmittel,Nuremberg,0,22761,61,0
733,P08734,31,W,Ukraine,Gesetzlich,Mittlere Reife,Neurologische Erkrankungen,Orthopaedie,Mittel,1,...,7,Russisch-Deutsch,7,13,Mietwagen,Berlin,0,14202,46,0
734,P08747,87,W,Russland,Gesetzlich,Abitur,Diabetes mellitus,Kardiologie,Niedrig,10,...,15,Russisch-Deutsch,9,15,Oeffentliche_Verkehrsmittel,Duisburg,1,29632,19,1


10. Revenue optimization analysis with price elasticity / Umsatzoptimierungsanalyse mit Preiselastizität

In [154]:
filtered = services.merge(
    treatment[
        (treatment["Status"] == "Abgeschlossen")
        & (treatment["Gesamtkosten_EUR"].notnull())
    ][["Leistungs_ID", "Gesamtkosten_EUR", "Patientenzufriedenheit"]],
    on="Leistungs_ID",
)

filtered["price_quintile"] = filtered.groupby("Kategorie")[
    "Gesamtkosten_EUR"
].transform(lambda x: pd.qcut(x, 5, labels=False, duplicates="drop") + 1)

price_segments = filtered.groupby(
    ["Kategorie", "Leistungsname", "price_quintile"], as_index=False
).agg(
    treatment_count=("Gesamtkosten_EUR", "count"),
    avg_price=("Gesamtkosten_EUR", "mean"),
    total_revenue=("Gesamtkosten_EUR", "sum"),
    satisfaction_rate=("Patientenzufriedenheit", lambda x: (x == "Zufrieden").mean()),
)

final = price_segments.groupby(["Kategorie", "price_quintile"], as_index=False).agg(
    service_count=("Leistungsname", "count"),
    total_treatments=("treatment_count", "sum"),
    avg_price_in_quintile=("avg_price", "mean"),
    quintile_revenue=("total_revenue", "sum"),
    avg_satisfaction_rate=("satisfaction_rate", "mean"),
)

final["revenue_per_treatment"] = final["quintile_revenue"] / final["total_treatments"]

final = final.sort_values(["Kategorie", "price_quintile"])

final

,Kategorie,price_quintile,service_count,total_treatments,avg_price_in_quintile,quintile_revenue,avg_satisfaction_rate,revenue_per_treatment
0,Augenheilkunde,1,9,15,9839.777778,147705,0.277778,9847.000000
1,Augenheilkunde,2,7,14,18741.023810,254170,0.000000,18155.000000
2,Augenheilkunde,3,6,14,27705.722222,381107,0.311111,27221.928571
3,Augenheilkunde,4,8,14,36859.093750,523677,0.093750,37405.500000
4,Augenheilkunde,5,8,14,45995.875000,643797,0.375000,45985.500000
5,Gastroenterologie,1,7,19,10363.571429,229325,0.271429,12069.736842
6,Gastroenterologie,2,8,19,23169.750000,443135,0.270833,23322.894737
7,Gastroenterologie,3,8,19,31871.231250,605116,0.231250,31848.210526
8,Gastroenterologie,4,6,19,35817.593254,686009,0.301587,36105.736842
9,Gastroenterologie,5,7,19,44425.228571,842890,0.128571,44362.631579


 11. Geographic clustering analysis with distance calculations / Geografische Clusteranalyse mit Distanzberechnung

In [156]:
patterns = patients.merge(
    treatment[
        [
            "Patienten_ID",
            "Leistungs_ID",
            "Gesamtkosten_EUR",
            "Begleitungstage",
            "Status",
        ]
    ],
    on="Patienten_ID",
).merge(services[["Leistungs_ID", "Kategorie"]], on="Leistungs_ID")
patterns = patterns[patterns["Status"] == "Abgeschlossen"]

patterns = patterns.groupby(["Herkunftsland", "Kategorie"], as_index=False).agg(
    treatment_frequency=("Leistungs_ID", "count"),
    avg_cost=("Gesamtkosten_EUR", "mean"),
    avg_stay_duration=("Begleitungstage", "mean"),
)

profiles = patterns.pivot_table(
    index="Herkunftsland",
    columns="Kategorie",
    values="treatment_frequency",
    fill_value=0,
).reset_index()

for cat in ["Kardiologie", "Onkologie", "Orthopaedie", "Neurologie"]:
    if cat not in profiles.columns:
        profiles[cat] = 0

profiles = profiles.rename(
    columns={
        "Kardiologie": "cardiology_freq",
        "Onkologie": "oncology_freq",
        "Orthopaedie": "orthopedics_freq",
        "Neurologie": "neurology_freq",
    }
)

profiles = profiles.merge(
    patterns.groupby("Herkunftsland", as_index=False).agg(
        overall_avg_cost=("avg_cost", "mean"),
        overall_avg_stay=("avg_stay_duration", "mean"),
    ),
    on="Herkunftsland",
)

results = []
for c1, c2 in combinations(profiles.itertuples(index=False), 2):
    dist = np.sqrt(
        (getattr(c1, "cardiology_freq", 0) - getattr(c2, "cardiology_freq", 0)) ** 2
        + (getattr(c1, "oncology_freq", 0) - getattr(c2, "oncology_freq", 0)) ** 2
        + (getattr(c1, "orthopedics_freq", 0) - getattr(c2, "orthopedics_freq", 0)) ** 2
        + (getattr(c1, "neurology_freq", 0) - getattr(c2, "neurology_freq", 0)) ** 2
    )
    cost_diff = abs(c1.overall_avg_cost - c2.overall_avg_cost)
    stay_diff = abs(c1.overall_avg_stay - c2.overall_avg_stay)
    results.append(
        {
            "country1": c1.Herkunftsland,
            "country2": c2.Herkunftsland,
            "treatment_pattern_distance": dist,
            "cost_difference": cost_diff,
            "stay_difference": stay_diff,
        }
    )

result_df = pd.DataFrame(results).sort_values("treatment_pattern_distance")

result_df

,country1,country2,treatment_pattern_distance,cost_difference,stay_difference
19,Estland,Lettland,1.000000,1573.204167,1.310417
2,Armenien,Georgien,3.000000,10018.097222,0.305556
39,Litauen,Moldawien,3.316625,1200.479167,1.072917
11,Belarus,Kasachstan,4.123106,1176.180434,0.662712
27,Georgien,Moldawien,4.242641,1328.253472,0.673611
35,Lettland,Litauen,4.582576,6086.489583,0.645833
20,Estland,Litauen,4.690416,4513.285417,0.664583
6,Armenien,Moldawien,5.196152,8689.843750,0.979167
26,Georgien,Litauen,5.385165,127.774306,1.746528
25,Georgien,Lettland,5.830952,6214.263889,1.100694


12. Advanced time series analysis with moving averages / Fortgeschrittene Zeitreihenanalyse mit gleitenden Durchschnitten

In [157]:
monthly_metrics = (
    treatment.merge(patients[["Patienten_ID", "Herkunftsland"]], on="Patienten_ID")
    .loc[
        treatment["Behandlungsdatum"].notnull()
        & (treatment["Status"] == "Abgeschlossen")
    ]
    .assign(month_year=lambda x: x["Behandlungsdatum"].dt.strftime("%Y-%m"))
    .groupby("month_year", as_index=False)
    .agg(
        monthly_treatments=("Behandlungsdatum", "count"),
        monthly_avg_cost=("Gesamtkosten_EUR", "mean"),
        monthly_revenue=("Gesamtkosten_EUR", "sum"),
        countries_served=("Herkunftsland", "nunique"),
    )
    .sort_values("month_year")
    .reset_index(drop=True)
)

monthly_metrics["ma3_treatments"] = (
    monthly_metrics["monthly_treatments"].rolling(window=3, min_periods=1).mean()
)
monthly_metrics["ma6_revenue"] = (
    monthly_metrics["monthly_revenue"].rolling(window=6, min_periods=1).mean()
)
monthly_metrics["prev_month_treatments"] = monthly_metrics["monthly_treatments"].shift(
    1
)

monthly_metrics["mom_growth_rate"] = (
    100
    * (monthly_metrics["monthly_treatments"] - monthly_metrics["prev_month_treatments"])
    / monthly_metrics["prev_month_treatments"]
).round(2)

monthly_metrics

,month_year,monthly_treatments,monthly_avg_cost,monthly_revenue,countries_served,ma3_treatments,ma6_revenue,prev_month_treatments,mom_growth_rate
0,2015-02,1,35556.000000,35556,1,1.000000,35556.000000,NaN,NaN
1,2015-03,4,27049.250000,108197,2,2.500000,71876.500000,1.0,300.00
2,2015-04,5,26210.800000,131054,5,3.333333,91602.333333,4.0,25.00
3,2015-05,8,29897.500000,239180,4,5.666667,128496.750000,5.0,60.00
4,2015-06,5,30979.600000,154898,2,6.000000,133777.000000,8.0,-37.50
...,...,...,...,...,...,...,...,...,...
115,2024-10,7,29550.142857,206851,3,7.000000,144595.000000,8.0,-12.50
116,2024-11,8,26883.750000,215070,6,7.666667,166511.833333,7.0,14.29
117,2024-12,5,21321.000000,106605,2,6.666667,171450.500000,8.0,-37.50
118,2025-01,8,25305.625000,202445,5,7.000000,178919.666667,5.0,60.00


13. Complex patient segmentation with RFM analysis / Komplexe Patientensegmentierung mit RFM-Analyse

In [158]:
rfm = (
    patients.merge(
        treatment[
            (treatment["Status"] == "Abgeschlossen")
            & (treatment["Behandlungsdatum"].notnull())
        ][["Patienten_ID", "Behandlungs_ID", "Behandlungsdatum", "Gesamtkosten_EUR"]],
        on="Patienten_ID",
    )
    .groupby(["Patienten_ID", "Herkunftsland", "Versicherungstyp"], as_index=False)
    .agg(
        recency_days=(
            "Behandlungsdatum",
            lambda x: (pd.Timestamp("2023-12-31") - x.max()).days,
        ),
        frequency=("Behandlungs_ID", "count"),
        monetary_value=("Gesamtkosten_EUR", "sum"),
        avg_treatment_value=("Gesamtkosten_EUR", "mean"),
    )
)

rfm["recency_score"] = (
    pd.qcut(rfm["recency_days"], 5, labels=False, duplicates="drop") + 1
)
rfm["frequency_score"] = (
    pd.qcut(
        rfm["frequency"].rank(method="first", ascending=False),
        5,
        labels=False,
        duplicates="drop",
    )
    + 1
)
rfm["monetary_score"] = (
    pd.qcut(
        rfm["monetary_value"].rank(method="first", ascending=False),
        5,
        labels=False,
        duplicates="drop",
    )
    + 1
)


def rfm_segment(row):
    r, f, m = row["recency_score"], row["frequency_score"], row["monetary_score"]
    if f >= 4 and m >= 4:
        return "Champions"
    elif r >= 3 and f >= 3 and m >= 3:
        return "Loyal Customers"
    elif r >= 4 and f <= 2:
        return "New Customers"
    elif r >= 3 and f <= 2 and m >= 3:
        return "Potential Loyalists"
    elif r <= 2 and f >= 3:
        return "At Risk"
    elif r <= 2 and f <= 2 and m >= 3:
        return "Cannot Lose Them"
    elif r <= 2 and f <= 2 and m <= 2:
        return "Lost Customers"
    else:
        return "Others"


rfm["rfm_segment"] = rfm.apply(rfm_segment, axis=1)

rfm_summary = (
    rfm.groupby(["rfm_segment", "Herkunftsland", "Versicherungstyp"], as_index=False)
    .agg(
        patient_count=("Patienten_ID", "count"),
        avg_recency=("recency_days", "mean"),
        avg_frequency=("frequency", "mean"),
        avg_monetary_value=("monetary_value", "mean"),
        total_revenue=("monetary_value", "sum"),
    )
    .sort_values("total_revenue", ascending=False)
)

rfm_summary

,rfm_segment,Herkunftsland,Versicherungstyp,patient_count,avg_recency,avg_frequency,avg_monetary_value,total_revenue
121,New Customers,Russland,Privat,31,2427.838710,1.064516,29353.322581,909953
120,New Customers,Russland,Gesetzlich,35,2436.457143,1.028571,25765.742857,901801
119,New Customers,Russland,Ausland,28,2595.214286,1.035714,30526.000000,854728
26,At Risk,Russland,Privat,22,366.090909,1.000000,35369.545455,778130
27,At Risk,Russland,Selbstzahler,17,300.529412,1.000000,38489.705882,654325
...,...,...,...,...,...,...,...,...
48,Champions,Lettland,Privat,1,826.000000,1.000000,8499.000000,8499
71,Loyal Customers,Armenien,Ausland,1,1590.000000,1.000000,8370.000000,8370
77,Loyal Customers,Georgien,Privat,1,1118.000000,1.000000,6841.000000,6841
43,Champions,Georgien,Privat,1,955.000000,1.000000,6734.000000,6734


14. Treatment pathway optimization with sequence mining / Optimierung von Behandlungspfaden mit Sequenzanalyse

In [159]:
df = treatment.merge(
    services[["Leistungs_ID", "Kategorie", "Leistungsname"]],
    on="Leistungs_ID",
).dropna(subset=["Behandlungsdatum"])

df = df.sort_values(["Patienten_ID", "Behandlungsdatum"])

df["sequence_order"] = df.groupby("Patienten_ID").cumcount() + 1
df["next_category"] = df.groupby("Patienten_ID")["Kategorie"].shift(-1)
df["next_treatment_date"] = df.groupby("Patienten_ID")["Behandlungsdatum"].shift(-1)

pathways = (
    df[df["next_category"].notnull()]
    .groupby(["Kategorie", "next_category"], as_index=False)
    .agg(
        transition_count=("Behandlungs_ID", "count"),
        avg_days_between=(
            "Behandlungsdatum",
            lambda x: (df.loc[x.index, "next_treatment_date"] - x).dt.days.mean(),
        ),
        avg_current_cost=("Gesamtkosten_EUR", "mean"),
        successful_transitions=("Status", lambda x: (x == "Abgeschlossen").sum()),
        failed_transitions=("Status", lambda x: (x == "Abgebrochen").sum()),
    )
)

pathways = pathways[pathways["transition_count"] >= 5].copy()
pathways = pathways.rename(columns={"Kategorie": "current_category"})

pathways["success_rate"] = (
    100
    * pathways["successful_transitions"]
    / (pathways["successful_transitions"] + pathways["failed_transitions"])
).round(2)

pathways["transition_probability"] = (
    100
    * pathways["transition_count"]
    / pathways.groupby("current_category")["transition_count"].transform("sum")
).round(2)

pathways = pathways.sort_values(
    ["current_category", "transition_count"], ascending=[True, False]
)

pathways

,current_category,next_category,transition_count,avg_days_between,avg_current_cost,successful_transitions,failed_transitions,success_rate,transition_probability
2,Augenheilkunde,Kardiologie,50,110.600000,28576.840000,7,0,100.0,13.77
7,Augenheilkunde,Radiologie,50,128.660000,27758.700000,2,0,100.0,13.77
5,Augenheilkunde,Onkologie,48,95.812500,25690.416667,5,0,100.0,13.22
0,Augenheilkunde,Augenheilkunde,47,122.148936,26423.297872,2,0,100.0,12.95
6,Augenheilkunde,Orthopaedie,46,98.586957,28381.565217,4,0,100.0,12.67
...,...,...,...,...,...,...,...,...,...
56,Radiologie,Augenheilkunde,55,92.872727,27724.854545,2,0,100.0,12.28
60,Radiologie,Neurologie,55,111.436364,26701.272727,5,0,100.0,12.28
63,Radiologie,Radiologie,53,99.773585,26230.245283,3,0,100.0,11.83
59,Radiologie,Labordiagnostik,46,111.391304,26487.217391,3,0,100.0,10.27


15. Predictive churn analysis with survival metrics / Prognostische Churn-Analyse mit Überlebensmetriken


In [160]:
activity = (
    patients.merge(
        treatment[(treatment["Status"] == "Abgeschlossen")][
            ["Patienten_ID", "Behandlungs_ID", "Behandlungsdatum", "Gesamtkosten_EUR"]
        ],
        on="Patienten_ID",
        how="left",
    )
    .groupby(
        [
            "Patienten_ID",
            "Anmeldedatum",
            "Herkunftsland",
            "Alter_bei_Anmeldung",
            "Versicherungstyp",
        ],
        as_index=False,
    )
    .agg(
        first_treatment=("Behandlungsdatum", "min"),
        last_treatment=("Behandlungsdatum", "max"),
        total_treatments=("Behandlungs_ID", "count"),
        total_spent=("Gesamtkosten_EUR", "sum"),
    )
)

activity = activity[activity["first_treatment"].notnull()].copy()

activity["days_since_last_treatment"] = (
    pd.Timestamp("2023-12-31") - activity["last_treatment"]
).dt.days
activity["churned"] = (activity["days_since_last_treatment"] > 365).astype(int)
activity["usage_pattern"] = np.select(
    [
        activity["total_treatments"] == 1,
        activity["total_treatments"].between(2, 5),
        activity["total_treatments"] > 5,
    ],
    ["One-time", "Occasional", "Frequent"],
    default=None,
)

churn_summary = activity.groupby(
    ["Herkunftsland", "Versicherungstyp", "usage_pattern"], as_index=False
).agg(
    patient_count=("Patienten_ID", "count"),
    churned_patients=("churned", "sum"),
    churn_rate=("churned", "mean"),
    avg_days_since_last_treatment=("days_since_last_treatment", "mean"),
    avg_lifetime_value=("total_spent", "mean"),
    avg_treatments_per_patient=("total_treatments", "mean"),
)

churn_summary["churn_rate"] = (churn_summary["churn_rate"] * 100).round(2)
churn_summary = churn_summary[churn_summary["patient_count"] >= 10]
churn_summary = churn_summary.sort_values("churn_rate", ascending=False)

churn_summary

,Herkunftsland,Versicherungstyp,usage_pattern,patient_count,churned_patients,churn_rate,avg_days_since_last_treatment,avg_lifetime_value,avg_treatments_per_patient
20,Kasachstan,Ausland,One-time,11,10,90.91,1443.272727,25645.454545,1.0
5,Belarus,Gesetzlich,One-time,10,9,90.00,1626.600000,25980.700000,1.0
25,Kasachstan,Selbstzahler,One-time,16,14,87.50,1266.875000,25253.812500,1.0
52,Ukraine,Selbstzahler,One-time,39,33,84.62,1599.564103,28141.179487,1.0
51,Ukraine,Privat,One-time,35,29,82.86,1435.971429,25588.085714,1.0
21,Kasachstan,Gesetzlich,One-time,11,9,81.82,1484.818182,25727.909091,1.0
47,Ukraine,Ausland,One-time,37,30,81.08,1200.351351,24555.783784,1.0
41,Russland,Gesetzlich,One-time,76,61,80.26,1499.223684,25418.789474,1.0
43,Russland,Privat,One-time,96,77,80.21,1267.270833,25938.750000,1.0
39,Russland,Ausland,One-time,77,61,79.22,1450.000000,27333.194805,1.0


## Master Level

1. Advanced machine learning feature engineering for patient outcome prediction / Fortgeschrittenes Feature Engineering für maschinelles Lernen zur Prognose von Behandlungsergebnissen

In [ ]:
country_risk = (
    patients.merge(treatment[["Patienten_ID", "Status"]], on="Patienten_ID")
    .groupby("Herkunftsland", as_index=False)
    .agg(risk_score=("Status", lambda x: (x == "Abgeschlossen").mean()))
)

patient_features = patients.merge(country_risk, on="Herkunftsland", how="left")
patient_features["risk_score"] = patient_features["risk_score"].fillna(0.5)
patient_features = patient_features.assign(
    is_male=(patient_features["Geschlecht"] == "M").astype(int),
    has_private_insurance=(patient_features["Versicherungstyp"] == "Privat").astype(
        int
    ),
    has_previous_treatment=(
        patient_features["Vorherige_Behandlung_Deutschland"] == "Ja"
    ).astype(int),
    family_cardiac_history=patient_features["Familienanamnese"]
    .str.contains("Herz|Kardio", case=False, na=False)
    .astype(int),
    family_cancer_history=patient_features["Familienanamnese"]
    .str.contains("Krebs|Tumor", case=False, na=False)
    .astype(int),
    education_level=np.select(
        [
            patient_features["Bildungsgrad"].isin(["Hochschule", "Universitaet"]),
            patient_features["Bildungsgrad"] == "Abitur",
            patient_features["Bildungsgrad"] == "Realschule",
        ],
        [3, 2, 1],
        default=0,
    ),
    country_risk_score=patient_features["risk_score"],
)

treat_complex = (
    treatment.merge(
        services[
            [
                "Leistungs_ID",
                "Kategorie",
                "Komplexitaet",
                "Dauer_Stunden",
                "Narkose_erforderlich",
            ]
        ],
        on="Leistungs_ID",
    )
    .merge(
        accompanying_staff[
            ["Mitarbeiter_ID", "Berufserfahrung_Jahre", "Sprachkombination"]
        ],
        on="Mitarbeiter_ID",
    )
    .merge(patients[["Patienten_ID", "Sprache"]], on="Patienten_ID")
)

treat_complex["complexity_score"] = np.select(
    [
        treat_complex["Komplexitaet"] == "Sehr hoch",
        treat_complex["Komplexitaet"] == "Hoch",
        treat_complex["Komplexitaet"] == "Mittel",
    ],
    [4, 3, 2],
    default=1,
)

treat_complex["duration_risk"] = np.select(
    [treat_complex["Dauer_Stunden"] > 10, treat_complex["Dauer_Stunden"] > 5],
    [3, 2],
    default=1,
)

treat_complex["anesthesia_risk"] = (
    treat_complex["Narkose_erforderlich"] == "Ja"
).astype(int)
treat_complex["staff_experience_factor"] = treat_complex["Berufserfahrung_Jahre"] / 20.0

treat_complex["language_barrier"] = treat_complex.apply(
    lambda row: int(
        (row["Sprache"] != "Deutsch")
        and (pd.notna(row["Sprachkombination"]))
        and (row["Sprache"] not in str(row["Sprachkombination"]))
    ),
    axis=1,
)

treat_complex["treatment_season"] = pd.cut(
    treat_complex["Behandlungsdatum"].dt.month,
    bins=[0, 2, 5, 8, 12],
    labels=["Winter", "Spring", "Summer", "Autumn"],
    right=True,
    include_lowest=True,
)

treat_complex["cost_percentile"] = treat_complex.groupby("Kategorie")[
    "Gesamtkosten_EUR"
].rank(pct=True)

features = treat_complex.merge(patient_features, on="Patienten_ID")

features = features.sort_values(["Patienten_ID", "Behandlungsdatum"])
features["prev_complexity"] = features.groupby("Patienten_ID")[
    "complexity_score"
].shift(1)
features["treatment_sequence_number"] = features.groupby("Patienten_ID").cumcount() + 1
features["age_complexity_interaction"] = (
    features["Alter_bei_Anmeldung"] * features["complexity_score"]
)
features["country_duration_risk"] = (
    features["country_risk_score"] * features["duration_risk"]
)
features["successful_outcome"] = (
    (features["Status"] == "Abgeschlossen")
    & (features["Patientenzufriedenheit"] == "Zufrieden")
).astype(int)

features = features[features["successful_outcome"].notnull()]
features = features.sort_values(["Patienten_ID", "treatment_sequence_number"])

features.head(10)

,Behandlungs_ID,Patienten_ID,Leistungs_ID,Mitarbeiter_ID,Anfragedatum,Beratungsdatum,Behandlungsdatum,Abschlussdatum,Status,Behandlungsort,...,has_previous_treatment,family_cardiac_history,family_cancer_history,education_level,country_risk_score,prev_complexity,treatment_sequence_number,age_complexity_interaction,country_duration_risk,successful_outcome
456,B00457,P00001,L00024,M005,2015-07-12,2015-07-18,2015-08-22,2015-08-28,Anreise_geplant,Munich,...,1,0,0,0,0.087369,NaN,1,26,0.087369,0
170,B00171,P00003,L00066,M010,2015-06-09,2015-06-11,2015-07-03,2015-07-16,Behandlung_abgeschlossen,Munich,...,1,0,0,2,0.081448,NaN,1,159,0.081448,0
479,B00480,P00004,L00038,M005,2015-05-03,2015-05-17,2015-07-02,2015-07-07,Storniert,Duisburg,...,1,0,0,0,0.075914,NaN,1,45,0.075914,0
294,B00295,P00004,L00025,M002,2015-12-28,2016-01-10,2016-02-19,2016-03-11,Angebot_erstellt,Duesseldorf,...,1,0,0,0,0.075914,3.0,2,30,0.151828,0
14,B00015,P00005,L00009,M005,2015-03-20,2015-03-31,2015-05-21,2015-05-22,Abgeschlossen,Duesseldorf,...,0,0,0,2,0.087369,NaN,1,141,0.087369,1
205,B00206,P00006,L00008,M004,2015-07-14,2015-07-16,2015-08-01,2015-08-05,Beratung_geplant,Leipzig,...,1,0,0,2,0.087369,NaN,1,44,0.262107,0
588,B00589,P00007,L00042,M005,2015-08-18,2015-08-28,2015-09-24,2015-10-01,Angebot_erstellt,Leipzig,...,0,0,0,2,0.087369,NaN,1,51,0.174738,0
297,B00298,P00009,L00036,M001,2015-01-10,2015-01-23,2015-03-12,2015-04-07,Angebot_erstellt,Duesseldorf,...,1,0,0,0,0.075914,NaN,1,27,0.151828,0
504,B00505,P00009,L00008,M008,2015-02-28,2015-03-05,2015-04-10,2015-04-30,In_Behandlung,Leipzig,...,1,0,0,0,0.075914,1.0,2,27,0.227741,0
9,B00010,P00010,L00058,M002,2015-06-06,2015-06-20,2015-08-14,2015-09-03,Anfrage,Frankfurt,...,1,0,0,2,0.081448,NaN,1,69,0.081448,0


2. Dynamic pricing optimization with elasticity analysis / Dynamische Preisoptimierung mit Elastizitätsanalyse

In [175]:
df = services.merge(
    treatment[
        (treatment["Behandlungsdatum"] >= "2020-01-01")
        & (treatment["Status"].isin(["Abgeschlossen", "Abgebrochen"]))
    ][
        [
            "Leistungs_ID",
            "Gesamtkosten_EUR",
            "Patientenzufriedenheit",
            "Status",
            "Behandlungsdatum",
            "Patienten_ID",
        ]
    ],
    on="Leistungs_ID",
).merge(
    patients[
        ["Patienten_ID", "Alter_bei_Anmeldung", "Versicherungstyp", "Herkunftsland"]
    ],
    on="Patienten_ID",
)

df["price_decile"] = df.groupby("Kategorie")["Gesamtkosten_EUR"].transform(
    lambda x: pd.qcut(x.rank(method="first"), 10, labels=False, duplicates="drop") + 1
)

grouped = df.groupby(
    ["Kategorie", "Leistungsname", "price_decile"], as_index=False
).agg(
    demand_volume=("Gesamtkosten_EUR", "count"),
    avg_price=("Gesamtkosten_EUR", "mean"),
    price_stddev=("Gesamtkosten_EUR", "std"),
    satisfaction_rate=("Patientenzufriedenheit", lambda x: (x == "Zufrieden").mean()),
    completion_rate=("Status", lambda x: (x == "Abgeschlossen").mean()),
    summer_demand_ratio=(
        "Behandlungsdatum",
        lambda x: x.dt.month.isin([6, 7, 8]).mean(),
    ),
    avg_patient_age=("Alter_bei_Anmeldung", "mean"),
    private_insurance_pct=("Versicherungstyp", lambda x: (x == "Privat").mean() * 100),
    country_diversity=("Herkunftsland", "nunique"),
)

cat_comp = (
    df.groupby("Kategorie")["Leistungsname"]
    .count()
    .reset_index(name="category_competition")
)
grouped = grouped.merge(cat_comp, on="Kategorie")

grouped = grouped.sort_values(["Kategorie", "Leistungsname", "price_decile"])
grouped["lag_demand"] = grouped.groupby(["Kategorie", "Leistungsname"])[
    "demand_volume"
].shift(1)
grouped["lag_price"] = grouped.groupby(["Kategorie", "Leistungsname"])[
    "avg_price"
].shift(1)

grouped["price_elasticity"] = (
    (grouped["demand_volume"] - grouped["lag_demand"]) * 100 / grouped["lag_demand"]
) / ((grouped["avg_price"] - grouped["lag_price"]) * 100 / grouped["lag_price"])

grouped["revenue"] = grouped["demand_volume"] * grouped["avg_price"]
grouped["quality_adjusted_revenue"] = grouped["revenue"] * grouped["satisfaction_rate"]

grouped["price_rank_in_category"] = grouped.groupby("Kategorie")["avg_price"].rank(
    method="min"
)
grouped["quality_rank_in_category"] = grouped.groupby("Kategorie")[
    "satisfaction_rate"
].rank(method="min", ascending=False)


def pricing_recommendation(row):
    if row["price_elasticity"] > -0.5 and row["satisfaction_rate"] > 0.8:
        return "Increase Price"
    elif row["price_elasticity"] < -2.0 and row["satisfaction_rate"] < 0.6:
        return "Decrease Price"
    elif -2.0 <= row["price_elasticity"] <= -0.5:
        return "Optimize Quality"
    else:
        return "Maintain Current Strategy"


def market_positioning(row):
    if row["price_rank_in_category"] <= 3 and row["quality_rank_in_category"] <= 3:
        return "Premium Positioning"
    elif row["price_rank_in_category"] >= 8 and row["quality_rank_in_category"] >= 8:
        return "Budget Positioning"
    else:
        return "Mid-Market Positioning"


grouped = grouped[grouped["price_elasticity"].notnull()]

grouped["pricing_recommendation"] = grouped.apply(pricing_recommendation, axis=1)
grouped["market_positioning"] = grouped.apply(market_positioning, axis=1)

result = grouped.sort_values(["Kategorie", "revenue"], ascending=[True, False])

result.head(10)

,Kategorie,Leistungsname,price_decile,demand_volume,avg_price,price_stddev,satisfaction_rate,completion_rate,summer_demand_ratio,avg_patient_age,...,category_competition,lag_demand,lag_price,price_elasticity,revenue,quality_adjusted_revenue,price_rank_in_category,quality_rank_in_category,pricing_recommendation,market_positioning
27,Augenheilkunde,Traenenkanal-Operation,10,2,47459.5,136.471609,0.5,1.0,0.5,83.5,...,36,1.0,30097.0,1.733449,94919.0,47459.5,26.0,5.0,Maintain Current Strategy,Mid-Market Positioning
24,Augenheilkunde,Schielen-Korrektur,9,2,42381.5,20.506097,0.5,1.0,0.0,50.5,...,36,1.0,37546.0,7.764657,84763.0,42381.5,24.0,5.0,Maintain Current Strategy,Mid-Market Positioning
1,Augenheilkunde,Augenlid-Operation,8,2,41189.0,367.695526,0.5,1.0,0.0,63.0,...,36,2.0,14492.5,0.000000,82378.0,41189.0,23.0,5.0,Maintain Current Strategy,Mid-Market Positioning
17,Augenheilkunde,Makuladegeneration-Behandlung,8,2,39565.5,696.500179,0.0,1.0,0.0,58.0,...,36,1.0,34499.0,6.809237,79131.0,0.0,22.0,8.0,Maintain Current Strategy,Budget Positioning
15,Augenheilkunde,Makuladegeneration-Behandlung,5,2,28062.5,2738.624564,0.0,1.0,0.0,63.0,...,36,1.0,14759.0,1.109407,56125.0,0.0,14.0,8.0,Maintain Current Strategy,Budget Positioning
5,Augenheilkunde,Glaukom-Operation,10,1,49876.0,NaN,1.0,1.0,1.0,46.0,...,36,2.0,21124.5,-0.367363,49876.0,49876.0,28.0,1.0,Increase Price,Mid-Market Positioning
13,Augenheilkunde,Laser-Behandlung,10,1,47794.0,NaN,0.0,1.0,0.0,69.0,...,36,1.0,38642.0,0.000000,47794.0,0.0,27.0,8.0,Maintain Current Strategy,Budget Positioning
18,Augenheilkunde,Makuladegeneration-Behandlung,9,1,43477.0,NaN,0.0,1.0,0.0,51.0,...,36,2.0,39565.5,-5.057587,43477.0,0.0,25.0,8.0,Decrease Price,Budget Positioning
4,Augenheilkunde,Glaukom-Operation,4,2,21124.5,2409.112804,0.0,1.0,0.5,52.5,...,36,1.0,17701.0,5.170440,42249.0,0.0,10.0,8.0,Maintain Current Strategy,Budget Positioning
12,Augenheilkunde,Laser-Behandlung,7,1,38642.0,NaN,0.0,1.0,0.0,20.0,...,36,1.0,25883.0,0.000000,38642.0,0.0,21.0,8.0,Maintain Current Strategy,Budget Positioning
